# Ch 5 Geometric Deep Learning Models
기하학적 딥러닝 모델 

기하학적 딥 러닝 청사진의 다양한 인스턴스(도메인, 대칭 그룹, 로컬리티 개념(notions of locality)의 다양한 선택)를 철저히 연구한 결과, 이러한 처방을 적용하여 가장 인기 있는 딥 러닝 아키텍처를 얻을 수 있는 방법에 대해 논의할 준비가 되었습니다. 

다시 한 번 말씀드리지만, 이번 설명은 일반적인 순서(order of generality)를 엄격하게 따르지 않을 것입니다. 먼저 컨볼루션 신경망(convolutional neural networks)(CNNs), 그룹 등변량(group-equivariant) CNNs, 그래프 신경망(graph neural networks)(GNNs) 등 구현이 거의 직접적으로 이어지는 세 가지 아키텍처를 다룹니다.

그런 다음 그래프 구조를 미리 알 수 없는 경우(즉, 정렬되지 않은 집합(sets))에 대한 GNN의 변형에 대해 자세히 살펴보고, 논의를 통해 널리 사용되는 Deep Sets과 Transformer 아키텍처를 GNN의 예로 설명하겠습니다.

기하학적 그래프와 메시를 살펴본 다음, 먼저 명시적인 기하학적 대칭을 GNN 계산에 도입하는 등변량 메시지 전달 네트워크(equivariant message passing networks)에 대해 설명합니다. 그런 다음, 측지(geodesics) 및 게이지(gauge) 대칭 이론을 딥 러닝 내에서 구체화하여 내재적 메시 CNN 제품군(측지(Geodesic) CNN, MoNet 및 게이지 등변수 메시(gauge-equivariant mesh ) CNN 포함)을 복구하는 방법을 보여줍니다. 

마지막으로 그리드 도메인을 시간적(temporal) 관점에서 되돌아봅니다. 이 논의는 순환신경망(recurrent neural networks)(RNN)으로 이어질 것입니다. 시간적 그리드에 대해 변환 등가성(translation equivariant)을 갖는 RNN의 방식을 보여줄 뿐만 아니라, 시간 왜곡(time warping) 변환에 대한 안정성도 연구할 것입니다. 이 속성은 장거리 종속성을 적절히 처리하는 데 매우 바람직하며, 이러한 변환에 클래스 불변성(invariance)을 적용하면 게이트 RNN의 클래스(LSTM 또는 GRU와 같은 널리 사용되는 RNN 모델 포함)를 정확히 산출할 수 있습니다.

이 글을 쓰는 시점에 사용 중인 대부분의 주요 딥 러닝 아키텍처를 위에서 설명했지만, 매일 새로운 신경망 인스턴스가 제안되고 있다는 것을 잘 알고 있습니다. 따라서, 가능한 모든 아키텍처를 다루기보다는 독자가 불변성(invariances)과 대칭(symmetries)이라는 렌즈를 사용해 향후 기하학적 딥 러닝 개발을 쉽게 분류할 수 있을 정도로 다음 섹션에서 충분히 예시적으로 설명했으면 합니다.

## 5.1 컨볼루션 신경망
Convolutional Neural Networks

컨볼루션 신경망은 3.5절에서 설명한 기하학적 딥러닝의 청사진을 따르는 딥러닝 아키텍처의 가장 초창기이자 가장 잘 알려진 예일 것입니다. 4.2절에서는 국부 필터 $θ$가 있는 컨볼루션 $\mathbf{C(θ)x = x \star θ}$로 주어지는 선형 및 국부 변환 등변수(translation equivariant) 연산자 클래스를 완전히 특성화했습니다. 먼저 스칼라 값('단일 채널' 또는 '그레이 스케일') 이산화된 이미지에 초점을 맞추겠습니다. 여기서 도메인은 그리드 $Ω = [H] × [W]$이고, $\mathbf{u} = (u_1, u_2), \mathbf{x} ∈ \mathcal{X} (Ω, \mathbb{R})$입니다.

>$C(θ)$는 매개변수 $θ$가 있는 순환(circulant) 행렬이라는 것을 기억하세요. 

컴팩트하게 지원되는 $H^f × W^f$ 크기의 필터를 사용하는 모든 컨볼루션은 제너레이터 $θ_{1,1}, ... . , θ_{H^f, W^f}$ 의 선형 조합으로 작성할 수 있으며, 단위 피크 $θ_{vw}(u_1, u_2) = δ(u_1 - v, u_2 - w)$로 표시됩니다. 따라서 모든 로컬 선형 등변량 맵은 다음과 같이 표현할 수 있습니다. 

$$
    \mathbf{F(x)} = \sum^{H^f}_{v=1} \sum^{W^f}_{w=1}α_{vw} \mathbf{C(θ}_{vw})\mathbf{x}, \tag{26}
$$

좌표로 표시되며, 이는 익숙한 2D 컨볼루션에 해당합니다(개요는 그림 14 참조): 

$$
    \mathbf{F(x)}_{uv} = \sum^{H^f}_{a=1} \sum^{W^f}_{b=1} α_{ab} x_{u+a, v+b}. \tag{27}
$$
.

>일반적으로 $x$와 $θ_{vw}$를 2D 행렬로 생각하지만, 이 방정식에서는 $x$와 $θ_{vw}$의 두 좌표 차원이 모두 하나로 평활화(flattened)되어 $x$는 벡터가 되고 $C(θ_{vw})$는 행렬이 된다는 점에 유의하세요.

기저(basis) $θ_{vw}$의 다른 선택도 가능하며 동등한(equivalent) 연산을 산출합니다($α_{vw}$의 잠재적으로 다른 선택의 경우). 흔히 사용되는 예는 방향 도함수(directional derivatives)입니다: $θ_{vw}(u_1, u_2) = δ(u_1, u_2)-δ(u_1-v, u_2-w),(v, w) \neq (0, 0)$을 국부 평균 $θ_0(u_1, u_2) = \frac{1}{H_fW_f}$ 와 함께 취합니다. 실제로 방향 도함수는 그래프에서 확산 과정의 그리드별(grid-specific) 유사체로 간주할 수 있으며, 각 픽셀을 그리드에서 바로 인접한 픽셀에 연결된 노드라고 가정하면 이를 복구할 수 있습니다.

스칼라 입력 채널이 여러 채널(예: RGB 색상 또는 더 일반적으로 임의의 수의 피처 맵(feature maps))로 대체되면, 컨볼루션 필터는 입력 피처의 임의의 선형 조합을 출력 피처 맵으로 표현하는 컨볼루션 텐서(convolutional tensor)가 됩니다. 좌표에서는 다음과 같이 표현할 수 있습니다. 

$$
   \mathbf{F(x)}_{uvj} = \sum^{H^f}_{a=1} \sum^{W^f}_{b=1} \sum^{M}_{c=1} 
α_{jabc}x_{u+a,v+b,c }, j ∈ [N], \tag{28}
$$

여기서 $M$과 $N$은 각각 입력 및 출력 채널의 수입니다. 이 기본 연산은 광범위한 종류의 신경망 아키텍처를 포함하며, 다음 섹션에서 설명하겠지만 컴퓨터 비전, 신호 처리 등 여러 분야에 걸쳐 큰 영향을 미쳤습니다. 여기서는 CNN의 무수히 많은 아키텍처 변형을 분석하기보다는, 광범위한 사용을 가능하게 한 몇 가지 필수적인 혁신에 초점을 맞추고자 합니다. 

### 효율적인 멀티스케일 컴퓨팅 
Efficient multiscale computation

일반 대칭에 대한 GDL 템플릿에서 설명한 것처럼, 컨볼루션 연산자 F에서 변환 불변 특징(translation invariant features)을 추출하려면 비선형 단계가 필요합니다. 컨볼루션 특징은 입력에 요소 단위로 작용하는 비선형 활성화 함수(activation function) σ를 통해 처리됩니다. 즉, $σ : \mathcal{X} (Ω) → \mathcal{X} (Ω)$, 즉 $σ(\mathbf{x})(u) = σ(\mathbf{x}(u))$로 처리됩니다. 이 글을 쓰는 시점에서 가장 많이 사용되는 예는 $σ(x) = \max(x, 0)$인 정류된 선형 단위(ReLU)일 것입니다. 이 비선형성은 신호를 효과적으로 정류(rectifies)하여 에너지를 낮은 주파수로 밀어내고, 구조를 반복하여 스케일에 걸친 고차 상호 작용을 계산할 수 있게 해줍니다.

>흔히 '현대적인' 아키텍처로 간주되는 ReLU는 이미 네오코그니트론(Neocognitron)에 사용되었습니다(후쿠시마와 미야케, 1982). 정류(Rectification)는 FM 라디오와 같은 많은 전송 프로토콜의 기초로서 전기 공학에서 기본이 되는 복조 원리와 동일하며, 신경 활동 모델에서도 중요한 역할을 합니다.

후쿠시마와 미야케(Fukushima and Miyake)(1982)와 르쿤(LeCun)(1998)의 초기 연구에서 이미 CNN과 유사한 아키텍처는 각 컨볼루션 레이어(28) 후에 그리드 거칠기 $\mathbf{P} : \mathcal{X} (Ω) → \mathcal{X} (Ω')$, 여기서 그리드 $Ω'$은 $Ω$보다 더 거친 해상도를 갖는 멀티스케일 구조를 가졌습니다. 이를 통해 수신 필드를 효과적으로 증가시키면서도, 스케일당 파라미터 수를 일정하게 유지하면서 멀티스케일 필터를 사용할 수 있습니다. 여러 가지 신호 조도(coarsening) 조정 전략 $\mathbf{P}$(풀링이라고 함)을 사용할 수 있으며, 가장 일반적인 방법은 로우패스 안티앨리어싱(lowpass anti-aliasing) 필터(예: 로컬 평균)를 적용한 후 그리드 다운샘플링(grid downsampling) 또는 비선형 최대 풀링(non-linear max-pooling)을 적용하는 것입니다. 

요약하면, '바닐라' CNN 레이어는 기하학적 딥러닝 청사진에서 이미 소개한 기본 오브젝트의 구성으로 표현할 수 있습니다: 

$$
    \mathbf{h = P(}σ\mathbf{(F(x)))}, \tag{29}
$$

즉, 등변량(equivariant) 선형 레이어 $\mathbf{F}$, 거칠기(coarsening) 연산 $\mathbf{P}$, 비선형성 $σ$. CNN 내에서 번역 불변 글로벌 풀링(translation invariant global pooling) 연산을 수행할 수도 있습니다. 직관적으로 이 작업은 각 픽셀이 여러 컨볼루션을 거친 후, 그 픽셀을 중심으로 패치(patch)를 요약하여 이미지의 최종 표현을 제안하고, 최종 선택은 이러한 제안의 합산 형태로 이루어집니다. 평균 함수는 이미지 크기에 관계없이 출력 크기가 비슷하게 유지되기 때문에, 일반적으로 많이 사용됩니다(Springenberg et al., 2014). 

이 CNN 청사진을 따르는 대표적인 예는 그림 15에 나와 있습니다(이 중 일부는 다음에 설명하겠습니다).

>이 단락에서 언급한 연산으로만 구성된 CNN을 흔히 "합성곱 (all-convolutional)"이라고 부릅니다. 이와는 대조적으로, 많은 CNN은 충분한 등변량(equivariant) 및 조도(coarsening) 조정 레이어가 적용된 후 공간 축(spatial axes)에 걸쳐 이미지를 평탄화(flatten)하여 MLP 분류기로 전달합니다. 이렇게 하면 번역 불변성(translation invariance)이 손실됩니다.

### 심층 및 잔여 네트워크
Deep and Residual Networks 

따라서 가장 간단한 형태의 CNN 아키텍처는 하이퍼파라미터 $(H^f_k, W^f_k, N_k, p_k)_{k≤K}$로 지정되며, 여기서 $M_{k+1} = N_k, p_k = 0, 1$은 그리드 조도 조정(coarsening)이 수행되는지 아닌지를 나타냅니다. 이러한 모든 하이퍼파라미터가 실제로 중요하지만, 특히 중요한 질문은 CNN 아키텍처에서 깊이 $K$의 역할과 이러한 주요 하이퍼파라미터를 선택할 때, 특히 필터 크기$(H^f_k, W^f_k)$와 관련하여 어떤 근본적인 트레이드오프가 발생하는지 이해하는 것입니다.

이 질문에 대한 엄격한 해답은 여전히 찾기 어렵지만(elusive), 최근 몇 년 동안 수집된 경험적 증거는 더 깊고(큰 $K$) 더 얇은(작은$(H^f_k, W^f_k)$) 모델에 유리한 트레이드 오프가 있다는 것을 시사합니다. 이러한 맥락에서 He 등(2016)의 중요한 통찰은 일반적인 비선형 변환이 아닌 이전 특징의 섭동(perturbation)을 모델링하기 위해 각 컨볼루션 층을 다시 파라미터화하는 것이었습니다:

$$
    \mathbf{h = P (x} + σ(\mathbf{F(x)))} . \tag{30}
$$
.

>역사적으로 ResNet 모델은 고속도로 네트워크(highway
networks)에 의해 선행되었으며(Srivastava 외., 2015), 이를 통해 보다 일반적인 게이팅 메커니즘으로 잔여 정보 흐름(residual information flow)을 제어(gating)할 수 있습니다.

이렇게 생성된 잔여(residual) 네트워크는 이전 공식에 비해 몇 가지 주요 이점을 제공합니다. 본질적으로 잔여 매개변수화는 심층 네트워크가 일반 미분 방정식(ordinary differential equation)(ODE)으로 모델링된 기본 연속 동역학 시스템(continuous dynamical system)의 이산화(discretisation)라는 관점과 일치합니다. 결정적으로, 속도를 모델링하여 동적 시스템을 학습하는 것은 위치를 직접 학습하는 것보다 훨씬 쉬운 것으로 밝혀졌습니다. 우리의 학습 설정에서, 이는 더 선호되는 기하학적 구조(favorable geometry)를 가진 최적화 환경(optimisation landscape)으로 해석되어, 이전보다 훨씬 더 심층적인 아키텍처를 학습할 수 있게 되었습니다. 향후 작업에서 설명하겠지만, 심층 신경망을 사용한 학습은 비볼록(non-convex) 최적화 문제를 정의하며, 이는 특정 단순화 체제(simplifying regimes)에서 경사 하강 방법을 사용하여 효율적으로 해결할 수 있습니다. ResNet 매개변수화의 주요 장점은 간단한 시나리오에서 엄격하게 분석되었으며(Hardt and Ma, 2016), 여전히 활발한 이론적 연구 분야로 남아 있습니다. 마지막으로, 신경 ODE(Neural ODEs)(Chen et al., 2018)는 최근 널리 사용되는 아키텍처로, ODE $\mathbf{\dot{x}} = σ(\mathbf{F(x)})$의 파라미터를 직접 학습하고 표준 수치 적분에 의존함으로써 ODE와의 유추를 한층 더 발전시킵니다.

>이 경우 ResNet은 ODE의 포워드 오일러 이산화(discretisation)를 수행합니다: $\mathbf{\dot{x}} = σ(\mathbf{F(x)})$

### 정규화
Normalisation 

CNN의 경험적 성능을 크게 향상시킨 또 다른 중요한 알고리즘 혁신은 정규화(normalisation) 개념입니다. 초기 신경 활동 모델에서는 뉴런이 일종의 국소적인 '이득 제어(gain control)'를 수행한다는 가설을 세웠는데, 여기서 층 계수(layer coefficients) $\mathbf{x}_k$는 $\mathbf{\tilde{x}}_k = σ^{-1}_k(\mathbf{x}_k - µ_k)$로 대체됩니다. 여기서, $µ_k$와 $σ_k$는 각각 $\mathbf{x}_k$의 1차 및 2차 모멘트 정보를 인코딩합니다. 또한 전역적으로 또는 로컬로 계산할 수 있습니다.

딥러닝의 맥락에서 이 원칙은 일괄 정규화(batch normalisation) 계층을 통해 널리 채택되었으며(Ioffe and Szegedy, 2015), 이후 여러 가지 변형이 이어졌습니다(Ba et al., 2016; Salimans and Kingma, 2016; Ulyanov et al., 2016; Cooijmans et al., 2016; Wu and He, 2018). 더 나은 최적화 환경이라는 측면에서 정규화의 이점을 엄격하게 설명하려는 일부 시도에도 불구하고(Santurkar et al., 2018), 이 글을 쓰는 시점에서 지침 원칙을 제공할 수 있는 일반 이론은 여전히 부재합니다.

>신경망의 활성화를 정규화하는 것은 일괄 정규화가 등장하기 이전부터 주목을 받아왔다는 점에 주목합니다. 예를 들어, Lyu와 Simoncelli(2008)를 참조하세요. 

### 데이터 증강
Data augmentation

CNN은 변환 불변성(translation invariance) 및 스케일 분리(scale separation)와 관련된 기하학적 전치(priors)를 인코딩하지만, 밝기(lightning)나 색상 변화, 작은 회전(rotations) 및 확대(dilations)와 같이 의미(semantic) 정보를 보존하는 다른 알려진 변환을 명시적으로 설명하지는 않습니다. 최소한의 아키텍처 변경으로 이러한 선구자를 통합하는 실용적인 접근 방식은 데이터 증강을 수행하는 것으로, 입력 이미지에 수동으로 이러한 변환을 수행하여 학습 세트에 추가하는 것입니다. 

데이터 증강은 경험적으로 성공적이었으며, 최첨단 비전 아키텍처를 훈련하는 것뿐만 아니라 자기 감독 및 인과적 표현 학습의 여러 발전을 뒷받침하기 위해 널리 사용되고 있습니다(Chen et al., 2020; Grill et al., 2020; Mitrovic et al., 2020). 그러나 샘플 복잡성(sample complexity) 측면에서 볼 때 이 방법은 최적이 아닌 것으로 입증되었습니다(Mei et al., 2021). 보다 효율적인 전략은 다음에서 논의하겠지만, 불변성(invariance) 그룹이 더 풍부한 아키텍처를 고려하는 것입니다.

## 5.2 그룹 등변량 CNN
Group-equivariant CNNs

4.3절에서 설명한 것처럼 컨볼루션 연산을 유클리드 공간(Euclidean space)의 신호에서 그룹 $\mathfrak{G}$가 작용하는 어떤 동질 공간(homogeneous space) $Ω$의 신호로 일반화할 수 있습니다. 변환된(translated) 필터가 신호와 일치하는 유클리드 컨볼루션과 유사하게, 그룹 컨볼루션의 개념은 회전(rotating) 및 변환(translating)과 같은 그룹 작업을 사용하여 도메인에서 필터를 이동하는 것입니다. 그룹 동작의 전이성(transitivity) 덕분에 필터를 $Ω$의 어느 위치로든 이동할 수 있습니다. 이 섹션에서는 구현 측면과 아키텍처 선택 등 그룹 컨볼루션의 일반적인 개념에 대한 몇 가지 구체적인 예를 살펴보겠습니다. 

>동질 공간(homogeneous space)은 전이적(transitive) 그룹 작용을 갖춘 집합 $Ω$으로, 모든 $u, v ∈ Ω$에 대해 $\mathfrak{g}.u = v$가 되는 $\mathfrak{g ∈ G}$가 존재한다는 것을 상기하십시오.

### 이산 그룹 컨볼루션 
Discrete group convolution

먼저 도메인 $Ω$과 그룹 $\mathfrak{G}$가 이산(discrete)인 경우를 고려합니다. 첫 번째 예로, 이산 이동(discrete translation) 및 회전 대칭을 가진 3D 그리드 상의 신호로 표현되는 의료용 체적(volumetric) 이미지를 살펴보겠습니다. 도메인은 3D 입방(cubical) 그리드 $Ω = \mathbb{Z}^3$이고 이미지(예: MRI 또는 CT 3D 스캔)는 $x : \mathbb{Z}^3 → \mathbb{R}, 즉 x ∈ \mathcal{X} (Ω)$ 함수로 모델링됩니다. 실제로 이러한 이미지는 유한 직육면체 $[W] × [H] × [D] ⊂ \mathbb{Z}^3$ 에서 지원되지만, 대신 적절한 제로 패딩이 있는 $\mathbb{Z}^3$ 의 함수로 보는 것을 선호합니다. 우리의 대칭의 관점에서, $\mathbb{Z}^3$ 에서의 거리 및 방향 보존 변환의 그룹 $\mathfrak{G} = \mathbb{Z}^3  \rtimes O_h$ 를 고려합니다. 이 그룹은 이동($\mathbb{Z}^3$)과 세 축에 대한 90도 회전으로 생성된 이산 회전(discrete rotations) Oh로 구성됩니다(그림 16 참조). 

두 번째 예로 C, G, A, T 네 개의 문자로 구성된 DNA 서열을 살펴봅시다: 이 염기서열은 1D 그리드 $Ω = \mathbb{Z}$에서 신호 $x : \mathbb{Z} → \mathbb{R}^4$ 로 나타낼 수 있으며, 각 문자는 $\mathbb{R}^4$ 에서 1-핫 코딩됩니다. 당연히 그리드에는 이산적인 1D 변환 대칭이 있지만, DNA 서열에는 흥미로운 대칭이 하나 더 있습니다. 이 대칭성은 DNA가 이중 나선으로 물리적으로 구현되는 방식과 세포의 분자 기계가 이를 읽는 방식에서 비롯됩니다. 이중 나선의 각 가닥은 5'-끝으로 시작하여 3'-끝으로 끝나며, 한 가닥의 5'은 다른 가닥의 3'으로 보완(complemented)됩니다. 즉, 두 가닥의 방향이 서로 반대입니다. DNA 분자는 항상 5'-끝에서 시작하여 읽혀지지만, 어느 쪽인지 알 수 없기 때문에, ACCCTGG와 같은 서열은 각 문자가 보체(complement)인 CCAGGGT로 대체된 역 서열과 동일합니다. 이를 문자 시퀀스의 역보완 대칭(reverse-complement symmetry)이라고 합니다. 따라서 아이덴티티 0과 역보완 변환(reverse-complement transformation) 1(및 구성 $1 + 1 = \mod 2$)에 해당하는 2원소 그룹 $\mathbb{Z}^2 = \{0, 1\}$이 있습니다. 전체 그룹은 변환과 역보완 변환을 결합합니다.

>DNA는 뉴클레오타이드(nucleotides)(시토신, 구아닌, 아데닌, 티민)라는 4개의 반복 단위로 이루어진 생체 고분자 분자로, 이중 나선으로 서로 감긴 두 가닥으로 배열되어 있으며, 각 뉴클레오타이드는 상보적인 것(염기쌍 A/T 및 C/G)의 반대편에 위치합니다.

>두 가닥이 파란색과 빨간색으로 표시된 DNA 이중 나선 구조의 모식도입니다. 나선의 염기서열이 상보적이며 역방향(5'에서 3'으로)으로 읽혀지는 점에 유의하세요. 

우리의 경우, 4.3절에서 정의한 그룹 컨볼루션(14)은 다음과 같이 주어집니다.

$$
    (x \star θ)(\mathfrak{g}) = \sum_{u∈Ω} x_uρ(\mathfrak{g})θ_u, \tag{31}
$$

는 (단일 채널) 입력 신호 $x$와 $ρ(\mathfrak{g})θ_u = θ_{\mathfrak{g}^{-1}u}$를 통해 $\mathfrak{g ∈ G}$로 변환된 필터 $θ$ 사이의 내적 곱(inner product)이며, 출력 $x \star θ$는 $\mathfrak{G}$의 함수입니다. $Ω$은 이산적이므로 방정식 (14)의 적분을 합으로 대체했습니다.

### 트랜스폼+컨볼브 접근 방식
Transform+Convolve approach

그룹 컨볼루션은 필터 변환(filter transformation) 단계와 번역 컨볼루션(translational convolution) 단계의 두 단계로 구현할 수 있음을 보여드리겠습니다. 필터 변환 단계는 기본 필터의 회전(rotated)(또는 역보완 변환(reverse-complement transformed)) 복사본을 생성하는 것으로 구성되며, 번역 컨볼루션(translational convolution)은 표준 CNN에서와 동일하므로 GPU와 같은 하드웨어에서 효율적으로 계산할 수 있습니다. 이를 확인하기 위해, 두 예제 모두에서 일반 변환 $\mathfrak{g ∈ G}$ 를 변환 $\mathfrak{h ∈ H}$ (예: 회전(rotation) 또는 역보완 변환(reverse-complement transformation))와 변환 $\mathfrak{k} ∈ \mathbb{Z}^d$ , 즉 $\mathbb{g = kh}$ (병치(juxtaposition)는 그룹 요소 $\mathbb{k}$ 와 $\mathbb{h}$ 의 구성을 나타냄)로 쓸 수 있다는 점에 유의하세요. 그룹 표현의 속성에 따라 $ρ(\mathfrak{g}) = ρ(\mathfrak{kh}) = ρ(\mathfrak{k})ρ(\mathfrak{h})$ 가 됩니다. 따라서

$$
\begin{align} \tag{32}
\begin{aligned} 
    (x \star θ)(\mathfrak{kh}) & = \sum_{u∈Ω} x_uρ(\mathfrak{k})ρ(\mathfrak{h})θ_u \\
    & = \sum_{u∈Ω} x_u(ρ(\mathfrak{h})θ)_{u-\mathfrak{k}}
\end{aligned}
\end{align}
$$

마지막 방정식은 신호 $x$ 와 변환된 필터 $ρ(\mathfrak{h})θ$ 의 표준(평면 유클리드) 컨볼루션으로 알아 볼 수 있습니다. 따라서, 이러한 그룹에 대한 그룹 컨볼루션을 구현하기 위해, 표준(canonical) 필터 $θ$ 를 취하고, 각 $\mathfrak{h ∈ H}$ 에 대해 변환된 복사본 $θ_\mathfrak{h} = ρ(\mathfrak{h})θ$ 를 생성한 다음(예: 각 회전 $\mathfrak{h} ∈ O_h$ 또는 역보완 DNA 대칭 $h ∈ \mathbb{Z}^2$ ), 이러한 각 필터로 $x$ 를 컨볼루션합니다: $(x \star θ)(\mathfrak{kh}) = (x \star θ_\mathfrak{h})(\mathfrak{k})$ . 두 예제 모두에서 대칭은 이산 회전에 대해, 그림 16에 표시된 것처럼, 필터 계수를 단순히 순열(permuting)하는 방식으로 필터에 작용합니다. 따라서 이러한 연산은 미리 계산된 인덱스가 있는 인덱싱 연산을 사용하여 효율적으로 구현할 수 있습니다. 

그룹 컨볼루션 $x \star θ$에 의해 출력되는 특징 맵을 $\mathfrak{G}$의 함수로 정의했지만, $\mathfrak{g}$를 $\mathfrak{h}$와 $\mathfrak{k}$로 나눌 수 있다는 사실은 필터 변환/방향 $\mathfrak{k}$당 하나의 특징 맵이 있는 유클리드 특징 맵(Euclidean feature maps)(방향 채널(orientation channels)이라고도 함)의 스택으로 생각할 수도 있다는 것을 의미합니다. 예를 들어 첫 번째 예에서는 각 필터 회전(그림 16의 각 노드)에 회전된 필터를 (전통적인 변환 의미에서) 컨볼루션하여 얻은 특징 맵을 연관시킬 수 있습니다. 따라서 이러한 특징 맵은 여전히 $W ×H ×C$ 배열로 저장할 수 있으며, 여기서 채널 수 $C$는 독립 필터 수에 변환 수 $\mathfrak{h ∈ H}$(예: 회전)를 곱한 값과 같습니다.

4.3절에 표시된 것처럼 그룹 컨볼루션은 $(ρ(\mathfrak{g})x) \star θ = ρ(\mathfrak{g})(x \star θ)$와 같이 등변량(equivariant)입니다. 이것이 방향 채널 측면에서 의미하는 바는 h의 작용에 따라, 각 방향 채널이 변환되고, 방향 채널 자체가 순열(permuted)된다는 것입니다. 예를 들어, 그림 16에서 변환당 하나의 방향 채널을 연결하고 z축을 중심으로 90도 회전(빨간색 화살표에 해당)을 적용하면, 빨간색 화살표와 같이 특징 맵이 순열됩니다. 이 설명을 통해 그룹 컨볼루션 신경망이 기존 CNN과 매우 유사하다는 것을 알 수 있습니다. 따라서 잔여(residual) 네트워크와 같이 5.1절에서 설명한 많은 네트워크 설계 패턴을 그룹 컨볼루션에도 사용할 수 있습니다. 

### 푸리에 영역의 구형 CNN
Spherical CNNs in the Fourier domain

4.3절에서 살펴본 구의 연속 대칭 그룹의 경우, 적절한 푸리에 변환을 사용하여 스펙트럼 영역에서 컨볼루션을 구현할 수 있습니다(S 2의 컨볼루션은 SO(3)의 함수이므로 다층 구형 CNN을 구현하려면 이 두 영역에서 푸리에 변환을 정의해야 함을 독자에게 상기시켜 드립니다). 구형 고조파(Spherical harmonics)는 2D 구에 대한 직교 기저로, 복소 지수(complex exponential)의 고전적인 푸리에 기저와 유사합니다. 특수 직교 그룹에서, 푸리에 기저는 위그너 D-함수(Wigner D-functions)로 알려져 있습니다. 두 경우 모두 푸리에 변환(계수)은 기저 함수의 내적 곱으로 계산되며, 컨볼루션 정리의 유추가 적용되어, 푸리에 변환의 요소별 곱으로 푸리에 영역에서 컨볼루션을 계산할 수 있습니다. 또한, S 2 및 SO(3)에서 푸리에 변환을 효율적으로 계산하기 위한 FFT와 유사한 알고리즘이 존재합니다. 자세한 내용은 Cohen et al. (2018)을 참고하시기 바랍니다.

## 5.3 그래프 신경망
Graph Neural Networks

그래프 신경망(GNN)은 순열(permutation) 그룹의 속성을 활용하여 그래프에서 기하학적 딥 러닝 청사진을 실현한 것입니다. GNN은 현재 존재하는 가장 일반적인 딥 러닝 아키텍처 클래스 중 하나이며, 이 글에서 살펴보겠지만 대부분의 다른 딥 러닝 아키텍처는 기하학적 구조를 추가한 GNN의 특수한 경우로 이해할 수 있습니다.

4.1절의 논의에 따라 인접 행렬(adjacency matrix) $\mathbf{A}$와 노드 특징(node features) $\mathbf{X}$로 지정된 그래프를 고려하고, 로컬 이웃에 공유된 순열 불변(permutation invariant) 함수 $φ(\mathbf{x}_u, \mathbf{X}_{\mathcal{N}_u} )$를 적용하여 구축된 순열 등변(permutation equivariant) 함수 $\mathbf{F(X, A)}$인 GNN 아키텍처를 연구하겠습니다. 이 국소 함수 φ는 다양한 이름으로 "확산(diffusion)", "전파(propagation)" 또는 "메시지 전달(message passing)"이라고 할 수 있으며, 이러한 F의 전체 계산을 "GNN 계층"이라고 부릅니다.

GNN 레이어의 설계와 연구는 이 글을 쓰는 시점에서 딥 러닝에서 가장 활발한 분야 중 하나이기 때문에 탐색하기 어려운 환경입니다. 다행히도 대부분의 문헌은 GNN 레이어의 세 가지 "맛(flavours)"(그림 17)에서만 파생될 수 있으며, 여기서는 이 세 가지 맛에 대해 소개합니다. 이 세 가지 유형은 $φ$가 이웃 피처(neighbourhood features)를 변형하는 정도를 결정하며, 그래프에서 상호작용을 모델링할 때 다양한 수준의 복잡성을 허용합니다.

세 가지 방식(flavour) 모두에서 순열 불변성(permutation invariance)은 (어떤 함수 ψ에 의해 잠재적으로 변형되있을 수 있는) $\mathbf{X}_{\mathcal{N}_u}$의 특징을 일부 순열 불변(permutation-invariant) 함수 $\bigoplus$로 집계한 다음, 일부 함수 $φ$에 의해 노드 $u$의 특징을 업데이트하고 이를 응집(aggregating)하여 보장됩니다. 일반적으로 $ψ$와 $φ$는 학습 가능한 반면, $\bigoplus$은 합, 평균 또는 최대와 같은 비모수적 연산으로 구현되지만 순환 신경망(RNN)을 사용하여 구축할 수도 있습니다(Murphy et al, 2018). 

>가장 일반적으로 $ψ$와 $φ$는 활성화 함수가 있는 학습 가능한 아핀(affine) 변환입니다(예: $ψ(x) = \mathbf{Wx + b}; φ(\mathbf{x, z}) = σ (\mathbf{Wx + Uz + b}))$. 여기서 $\mathbf{W, U, b}$는 학습 가능한 파라미터이고 $σ$는 정류된 선형 단위와 같은 활성화 함수입니다. $φ$에 대한 $x_u$의 추가 입력은 선택적 스킵 연결을 나타내며, 이는 종종 매우 유용합니다.

컨볼루션 방식(flavour)(Kipf and Welling, 2016a; Defferrard et al., 2016; Wu et al., 2019)에서는 이웃 노드의 특징이 고정 가중치로 직접 합산됩니다,

$$
    \mathbf{h}_u = φ \left( \mathbf{x}_u, \bigoplus_{v∈\mathcal{N}_u} c_{uv}ψ(\mathbf{x}_v)  \right). \tag{33}
$$

여기서 $c_uv$는 노드 $u$의 표현에 있어서 노드 $v$의 중요도를 지정합니다. 이는 종종 그래프의 구조를 나타내는 $\mathbf{A}$의 항목에 직접적으로 의존하는 상수입니다. 집계 연산자 $\bigoplus$을 합산으로 선택하면 컨볼루션의 일반화인 선형 확산(linear diffusion) 또는 위치 종속 선형 필터링(position-dependent linear filtering)으로 간주할 수 있다는 점에 유의하세요. 특히 4.4절과 4.6절에서 살펴본 스펙트럼 필터는 노드별 신호에 고정 로컬 연산자(예: 라플라시안 행렬)를 적용하는 것이므로 이 범주에 속합니다.

>이 방식(flavour)은 (그래프 구조와 통근(commuting)한다는 의미에서) 모든 합성곱 신경망 계층을 표현하지는 않지만, 실제로 제안된 대부분의 접근 방식을 포괄한다는 점에 유의할 필요가 있습니다. 향후 작업에서 자세한 논의와 확장을 제공할 예정입니다.

주의력(attentional) 풍미(flavour)(Veličković 외, 2018; Monti 외, 2017; Zhang 외, 2018)에서는 상호 작용이 암시적으로 이루어집니다.

$$
    \mathbf{h}_u = φ \left( \mathbf{x}_u, \bigoplus_{v∈\mathcal{N}_u} a(\mathbf{x}_u, \mathbf{x}_v)ψ(\mathbf{x}_v)  \right). \tag{34}
$$

여기서 $a$는 학습 가능한 자기 주의 메커니즘(self-attention mechanism)으로, 중요도 계수 $α_{uv} = a(\mathbf{x}_u, \mathbf{x}_v)$를 암시적으로 계산합니다. 이 계수는 종종 모든 이웃에 걸쳐 소프트 최대 정규화(softmax-normalised)됩니다. $\bigoplus$이 합계인 경우 집계는 여전히 이웃 노드 특징의 선형 조합이지만 이제 가중치는 특징에 따라 달라집니다(feature-dependent).

마지막으로, 메시지 전달(message-passing) 방식(flavour)(Gilmer et al., 2017; Battaglia et al., 2018)은 에지를 가로지르는 임의의 벡터("메시지(messages)")를 계산하는 방식입니다,

$$
    \mathbf{h}_u = φ \left( \mathbf{x}_u, \bigoplus_{v∈\mathcal{N}_u} ψ(\mathbf{x}_u, \mathbf{x}_v)  \right).\tag{35}
$$

여기서 $ψ$는 학습 가능한 메시지 함수(message function)로, $u$로 전송된 $v$의 벡터를 계산하며, 집계(aggregation)는 그래프에서 메시지 전달의 한 형태로 간주할 수 있습니다.

주목해야 할 한 가지 중요한 점은 컨볼루션 $⊆$ 주의 $⊆$ 메시지 전달 (convolution $⊆$ attention $⊆$ message-passing)이라는 이러한 접근 방식 사이의 표현적 포함관계(containment)입니다. 실제로 주의 GNN은 룩업 테이블 $a(\mathbf{x}_u, \mathbf{x}_v) = c_{uv}$ 로 구현된 주의 메커니즘으로 컨볼루션 GNN을 나타낼 수 있으며, 컨볼루션과 주의 GNN은 모두 메시지가 발신자 노드의 특징만 있는 메시지 전달의 특수한 경우입니다: 컨볼루션 GNN의 경우 $ψ(\mathbf{x}_u, \mathbf{x}_v) = c_{uv}ψ(\mathbf{x}_v)$, 어텐션션 GNN의 경우 $ψ(\mathbf{x}_u, \mathbf{x}_v) = a(\mathbf{x}_u, \mathbf{x}_v)ψ(\mathbf{x}_v)$가 됩니다. 

그렇다고 해서 메시지 전달 GNN이 항상 가장 유용한 변형이라는 의미는 아닙니다. 에지에서 벡터 값 메시지를 계산해야 하기 때문에 일반적으로 훈련하기가 더 어렵고 많은 양의 메모리를 필요로 합니다. 또한, 자연적으로 발생하는 광범위한 그래프에서 그래프의 에지는 다운스트림 클래스 유사성을 인코딩합니다(즉, 에지$(u, v)$는 $u$와 $v$가 동일한 출력을 가질 가능성이 높다는 것을 의미함). 이러한 그래프(흔히 동형 그래프(homophilous)라고 함)의 경우, 정규화 및 확장성 측면에서 인접 영역에 걸친 컨볼루션 집계(convolutional aggregation)가 훨씬 더 나은 선택인 경우가 많습니다. 주의적 GNN은 "중간 지점"을 제공합니다. 에지에 걸쳐 스칼라 값의 수량만 계산하면서 이웃 간의 복잡한 상호작용을 모델링할 수 있으므로 메시지 전달보다 확장성이 뛰어납니다. 

여기에 제시된 "세 가지 맛"의 분류는 간결성을 염두에 두고 제공된 것으로, GNN 모델에 대한 풍부한 뉘앙스, 통찰력, 일반화 및 역사적 맥락을 무시할 수밖에 없습니다. 중요한 점은, 이 분류에서는 Weisfeiler-Lehman 계층 구조에 기반한 고차원 GNN과 그래프 푸리에 변환의 명시적 계산에 의존하는 스펙트럼 GNN이 제외된다는 점입니다. 

## 5.4 딥 세트, 트랜스포머 및 잠재 그래프 추론
Deep Sets, Transformers, and Latent Graph Inference

정렬되지 않은 집합(unordered sets)의 표현을 학습하기 위한 순열 등변량(permutation-equivariant) 신경망 아키텍처에 대해 언급하면서 GNN에 대한 논의를 마무리합니다. 집합(sets)은 이 글에서 논의한 영역 중 가장 구조가 단순하지만, 최근 트랜스포머(Transformers)(Vaswani et al., 2017) 및 딥셋(Deep Sets)(Zaheer et al., 2017)과 같이 매우 인기 있는 아키텍처에 의해 그 중요성이 강조되고 있습니다. 4.1절의 언어에서는 노드 특징의 행렬인 $\mathbf{X}$가 주어지지만 노드 간의 인접성(adjacency)이나 순서 정보가 지정되지 않은 것으로 가정합니다. 구체적인 아키텍처는 노드 간의 상호작용(interactions)을 어느 정도까지 모델링할지 결정함으로써 발생합니다.

### 빈 가장자리 세트
Empty edge set

정렬되지 않은 집합은 추가적인 구조나 지오메트리 없이 제공되므로, 이를 처리하는 가장 자연스러운 방법은 각 집합 요소를 완전히 독립적(independently)으로 처리하는 것이라고 주장할 수 있습니다. 이는 4.1절에서 이미 소개한 바 있는 모든 노드에 개별적으로 적용되는 공유 변환인 이러한 입력에 대한 순열 등변수(permutation equivariant) 함수로 해석할 수 있습니다. GNN을 설명할 때와 동일한 표기법을 사용한다고 가정하면(5.3절), 이러한 모델은 다음과 같이 표현할 수 있습니다.

$$
    \mathbf{h}_u = ψ(\mathbf{x}_u),
$$

여기서 $ψ$는 학습 가능한 변환입니다. 이는 $\mathcal{N}_u = \{u\}$, 또는 이에 상응하는 $\mathbf{A = I}$를 갖는 컨볼루션 GNN의 특수한 경우임을 알 수 있습니다. 이러한 아키텍처는 이러한 아키텍처의 몇 가지 보편적 근사(universal-approximation) 특성을 이론적으로 증명한 Zaheer 등(2017)의 연구를 인정하여 일반적으로 딥셋(Deep Sets)이라고 불립니다. 컴퓨터 비전과 그래픽에서 포인트 클라우드(point clouds)를 처리할 때 정렬되지 않은 집합을 처리해야 할 필요성은 일반적으로 발생하며, 이러한 모델을 포인트넷(PointNets)이라고 합니다(Qi et al., 2017). 

### 완전한 엣지 세트
Complete edge set

빈 에지 집합을 가정하는 것은 정렬되지 않은 집합에 함수를 구축하는 데 매우 효율적인 구조이지만, 종종 집합의 요소들이 어떤 형태의 관계형 구조, 즉 노드 사이에 잠재 그래프(latent graph)가 존재할 것으로 예상할 수 있습니다. $\mathbf{A = I}$로 설정하면 이러한 구조가 무시되어 최적이 아닌 성능이 나올 수 있습니다. 반대로, 다른 사전 지식이 없는 경우, 노드 간의 가능한 링크를 미리 배제할 수 없다고 가정할 수 있습니다. 이 접근 방식에서는 완전한(complete) 그래프인 $\mathbf{A = 11}^{\intercal}$, 즉 $\mathcal{N}_u = \mathcal{V}$를 가정합니다. 상호 작용 계수에 대한 접근을 가정하지 않기 때문에 이러한 그래프에 대해 컨볼루션 유형의 GNN을 실행하면 다음과 같습니다: 

$$
    \mathbf{h}_u = φ \left( \mathbf{x}_u, \bigoplus_{v∈\mathcal{V}} ψ(\mathbf{x}_v)  \right),
$$

여기서 두 번째 입력인 $\bigoplus_{v∈\mathcal{V}} ψ(xv)$는 모든 노드 $u$에 대해 동일(identical)하므로 모델을 해당 입력을 완전히 무시하는 것과 동등하게 표현할 수 있습니다(즉, 위에서 언급한 $\mathbf{A = I}$의 경우).

>이는 L의 순열 불변성(permutation invariance)의 직접적인 결과입니다. 

이를 통해 보다 표현력이 풍부한 GNN 풍미인 주의력(attentional)을 사용하게 되었습니다, 

$$
    \mathbf{h}_u = φ \left( \mathbf{x}_u, \bigoplus_{v∈\mathcal{V}} a(\mathbf{x}_u, \mathbf{x}_v)ψ(\mathbf{x}_v)  \right). \tag{36}
$$

를 계산하면 트랜스포머(Transformer) 아키텍처의 핵심인 자기 주의(self-attention) 연산자가 산출됩니다(Vaswani et al., 2017). 주의 계수(예: 소프트맥스(softmax))에 대해 일종의 정규화를 가정하면, 모든 스칼라 $a(\mathbf{x}_u, \mathbf{x}_v)$가 $[0, 1]$ 범위에 있도록 제한할 수 있습니다. 따라서 자기 주의(self-attention)는 일부 다운스트림 작업(downstream task)에 대한 경사 기반 최적화의 부산물로서 소프트 인접 행렬(soft adjacency matrix) $a_{uv} = a(\mathbf{x}_u, \mathbf{x}_v)$를 추론하는 것으로 생각할 수 있습니다. 

위의 관점은 트랜스포머를 완전한 그래프(complete graph)에 대한 주의적 GNN으로 정확히 취(pose)할 수 있음을 의미합니다(Joshi, 2020). 그러나 이는 트랜스포머가 처음에 시퀀스(sequences) 모델링을 위해 제안된 것과는 명백히 상충되는 것으로, $h_u$의 표현은 시퀀스에서 노드 $u$의 위치를 염두에 두어야 하는데, 완전 그래프 집계는 이를 무시하기 때문입니다. 트랜스포머는 위치 인코딩(positional encodings)을 도입하여 이 문제를 해결합니다. 노드 특징 $\mathbf{x}_u$는 시퀀스에서 노드 $u$ 의 위치를 인코딩하도록 증강(augmented)되며, 일반적으로 주파수가 $u$에 의존하는 사인파의 샘플로 인코딩됩니다.

>메시지 전달 특성(flavour)을 적용하는 것도 적절합니다. 물리학 시뮬레이션과 관계 추론에 널리 사용되기는 하지만(예: Battaglia 외. (2016), Santoro 외. (2017)), 트랜스포머만큼 널리 사용되지는 않았습니다. 이는 완전한 그래프에서 벡터 메시지를 계산하는 것과 관련된 메모리 문제 또는 벡터 기반 메시지가 자기 주의(self-attention)가 제공하는 '소프트 인접성(soft adjacency)'에 비해 해석 가능성(less
interpretable)이 낮기 때문일 수 있습니다.

노드의 자연스러운 순서가 존재하지 않는 그래프에서는 이러한 위치 인코딩에 대한 여러 가지 대안이 제안되었습니다. 이러한 대안에 대한 논의는 나중으로 미루지만, 트랜스포머에서 사용되는 위치 인코딩이 이산 푸리에 변환(discrete Fourier transform)(DFT)과 직접적으로 관련될 수 있으며, 따라서 '원형 격자(circular grid)'의 라플라시안 그래프의 고유 벡터와도 관련될 수 있다는 사실을 깨닫는 것이 한 가지 유망한 방향이라는 점에 주목할 필요가 있습니다. 따라서 트랜스포머의 위치 인코딩은 입력 노드가 격자로 연결되어 있다는 가정을 암시적으로 표현하고 있습니다. 보다 일반적인 그래프 구조의 경우, (가정된) 그래프의 라플라시안 고유 벡터를 사용하면 되는데, 이는 Dwivedi와 Bresson(2020)이 경험적으로 강력한 그래프 트랜스포머(Graph Transformer) 모델에서 활용한 관찰 결과입니다.

### 추론된 에지 세트 
Inferred edge set

마지막으로, 잠재적(latent) 관계 구조를 학습하여 $\mathbf{I}$도 아니고 $\mathbf{11}^{\intercal}$도 아닌 일반 $\mathbf{A}$를 도출할 수 있습니다. GNN이 사용할 잠재 인접 행렬 $\mathbf{A}$를 추론하는 문제(종종 잠재 그래프 추론(latent graph inference)이라고도 함)는 그래프 표현 학습에 있어 매우 중요합니다. 이는 $\mathbf{A = I}$라고 가정하는 것이 표현력이 떨어질(inferior) 수 있고, $\mathbf{A = 11^{\intercal}}$은 메모리 요구 사항과 집계해야 할 넓은 이웃으로 인해 구현하기 어려울 수 있기 때문입니다. 또한, 인접 행렬 $\mathbf{A}$를 추론한다는 것은 $\mathbf{X}$의 행 사이에서 유용한 구조를 감지하는 것을 의미하며, 이는 변수 간의 인과 관계와 같은 가설을 세우는 데 도움이 될 수 있다는 점에서 "참" 문제에 가장 가깝습니다.

안타깝게도 이러한 프레임워크는 필연적으로 모델링 복잡성을 한 단계 끌어올릴 수밖에 없습니다. 특히, 그래프가 사용되는 모든 다운스트림 작업과 구조 학습 목표(structure learning objective)(불연속적(discrete)이기 때문에 그라데이션 기반 최적화에 어려움이 있음)의 균형을 적절히 맞춰야 합니다. 따라서 잠재 그래프 추론은 매우 까다롭고 복잡한 문제입니다.

## 5.5 등변량 메시지 전달 네트워크
Equivariant Message Passing Networks

그래프 신경망의 많은 응용 분야에서 노드 특징(또는 그 일부)은 임의의 벡터가 아니라 기하학적 엔티티의 좌표(coordinates)입니다. 예를 들어 분자 그래프를 다룰 때 원자를 나타내는 노드는 원자 유형에 대한 정보와 3D 공간 좌표를 포함할 수 있습니다. 분자가 공간에서 변형되는 것과 같은 방식으로, 즉 앞서 설명한 표준 순열 등식에 더해 강체 운동(rigid motions)(회전, 이동, 반사 ((rotations, translations, and reflections))의 유클리드 그룹 $E(3)$에 등가(equivariant)되는 방식으로 특징의 후반부를 처리하는 것이 바람직합니다. 

(약간 단순화된) 분석의 단계를 설정하기 위해, 노드 특징 $\mathbf{f}_u ∈ \mathbb{R}^d$와 노드 공간 좌표(spatial coordinates) $\mathbf{x}_u ∈ \mathbb{R}^3$을 구분하고, 후자는 유클리드 대칭 구조를 부여합니다. 이 설정에서 등변량(equivariant) 레이어는 이 두 입력을 명시적으로 개별적으로 변환하여 수정된 노드 특징 $f'_u$와 좌표 $x'_u$를 산출합니다.

이제 기하학적 딥러닝 청사진에 따라 바람직한 등분(equivariance) 속성을 명시할 수 있습니다. 입력의 공간 성분이 $\mathfrak{g} ∈ E(3)(ρ(\mathbf{\mathfrak{g})x = Rx + b}$로 표현되며, 여기서 $\mathbf{R}$은 회전과 반사를 모델링하는 직교 행렬이고 $\mathbf{b}$는 병진 벡터)로 변환되면 출력의 공간 성분도 같은 방식으로 변환되지만$(\mathbf{x'}_u \mapsto \mathbf{Rx'}_u + \mathbf{b}), f'_u$는 불변(invariant)으로 유지됩니다. 

앞서 일반 그래프의 맥락에서 논의한 순열 등변수(permutation equivariant) 함수의 공간과 마찬가지로, 위의 제약 조건을 만족하는 방대한 양의 $E(3)$-등변수 레이어가 존재하지만, 이러한 레이어가 모두 기하학적으로 안정적이거나 구현하기 쉬운 것은 아닙니다. 사실, 실제로 유용한 등변량(equivariant) 레이어의 공간은 공간 GNN 레이어의 "세 가지 맛(flavours)"과 달리 간단한 분류로 쉽게 설명할 수 있습니다. Satorras 등(2021)은 등변량 메시지 전달(equivariant message passing)의 형태로 한 가지 우아한 해결책을 제안했습니다. 그들의 모델은 다음과 같이 작동합니다:

$$
\begin{align}
    \mathbf{f}'_u & = φ \left( \mathbf{f}_u, \bigoplus_{v∈\mathcal{N}_u}
        ψ_f(\mathbf{f}_u,\mathbf{f}_v, \| \mathbf{x}_u − \mathbf{x}_v \|^2
\right),
\\
    \mathbf{x}'_u & = \mathbf{x}_u + \sum_{v \neq u} ( \mathbf{x}_u − \mathbf{x}_v ) ψ_c (\mathbf{f}_u,\mathbf{f}_v, \| \mathbf{x}_u − \mathbf{x}_v \|^2)    
\end{align}
$$

여기서 $ψ_f$와 $ψ_c$는 서로 다른 두 개의 (학습 가능한) 함수입니다. 이러한 집계는 공간 좌표의 유클리드 변환 하에서 등변수(equivariant)임을 알 수 있습니다. 이는 xu에 대한 $f'_u$의 유일한 의존성이 거리 $\| \mathbf{x}_u − \mathbf{x}_v \|^2$ 를 통해서만 발생하며, $E(3)$의 작용은 반드시 노드 간의 거리를 변경하지 않기 때문입니다. 또한 이러한 계층의 계산은 "메시지 전달(message passing)" GNN의 특정 인스턴스로 볼 수 있으므로 구현하기에 효율적입니다.

요약하자면, 일반 GNN과 달리 Satorras 등(2021)은 그래프의 각 점에 대한 '좌표(coordinates)'를 올바르게 처리할 수 있습니다. 이제 이들은 $E(3)$ 그룹의 멤버로 취급되며, 이는 네트워크 출력이 90° 입력의 회전, 반사 및 변환에서 올바르게 작동한다는 것을 의미합니다. 그러나 특징(features)인 $f_u$는 채널 단위로 처리되며 여전히 이러한 변환에서도 변하지 않는 스칼라로 간주됩니다. 따라서 이러한 프레임워크 내에서 캡처할 수 있는 공간 정보의 유형이 제한됩니다. 예를 들어, 일부 특징(예: 점 속도)은 이러한 변환에 따라 방향을 변경해야 하는 벡터로 인코딩하는 것이 바람직할 수 있습니다. Satorras 등(2021)은 아키텍처의 한 변형에 속도 개념을 도입하여 이 문제를 부분적으로 완화했습니다. 속도는 적절하게 회전하는 각 점의 3D 벡터 속성입니다. 그러나 이것은 $E(3)$ 등변량(equivariant) 네트워크로 학습할 수 있는 일반적인 표현의 작은 부분 공간에 불과합니다. 일반적으로 노드 특징(features)은 잘 정의된 방식으로 $E(3)$에 따라 변환되는 임의의 차원의 텐서를 인코딩할 수 있습니다. 

>스칼라 특징(히트맵)은 회전해도 변하지 않지만, 벡터 특징(화살표)은 방향이 바뀔 수 있습니다. 앞서 설명한 단순 $E(3)$ 등변량(equivariant) GNN은 이를 고려하지 않았습니다.

따라서 위에서 설명한 아키텍처는 이미 많은 실제 입력 표현에 대해 우아한 등변량(equivariant) 솔루션을 제공하지만, 경우에 따라서는 등변량 속성을 만족하는 더 광범위한 함수 모음을 탐색하는 것이 바람직할 수 있습니다. 이러한 설정을 다루는 기존 방법은 (앞서 언급한 레이어가 단순화된 인스턴스인) 환원 불가능한 표현(irreducible representations)과 일반 표현(regular representations)의 두 가지 클래스로 분류할 수 있습니다. 여기서는 간략하게 살펴보고 자세한 논의는 추후 작업으로 남겨두겠습니다.
	
### 환원 불가능한 표현
Irreducible representations

환원 불가능한(Irreducible) 표현은 로토(회전) 번역(roto-translation) 그룹의 모든 요소를 환원 불가능한 형태, 즉 블록 대각선(block diagonal) 행렬에 의해 회전되는 벡터로 가져올 수 있다는 발견에 기반합니다. 결정적으로, 이러한 각 블록은 위그너 D 행렬(Wigner D-matrix)(앞서 언급한 구형(Spherical) CNN의 푸리에 기저)입니다. 이 우산 아래에서 접근하는 방법은 등변량(equivariant) 커널을 사용하여 한 환원 불가능한 표현 집합에서 다른 표현 집합으로 매핑합니다. 등변량 매핑의 전체 집합을 찾으려면, 이러한 커널에 대한 등변량 제약 조건(equivariance constraint)을 직접 풀 수 있습니다. 이 해(solutions)는 클렙쉬-고단 행렬(Clebsch-Gordan matrices)과 구형 고조파(spherical harmonics)에 의해 유도된 등변수 기저 행렬(equivariant basis matrices)의 선형 조합을 형성합니다.

환원 불가능한 표현 접근법의 초기 사례로는 포인트 클라우드(point clouds)에서 작동하는 컨볼루션 모델인 텐서 필드 네트워크(Tensor Field Networks)(Thomas et al., 2018)와 3D 스티어러블 CNN(3D Steerable CNNs)(Weiler et al., 2018)이 있습니다. Fuchs 등(2020)의 SE(3)-트랜스포머(SE(3)-Transformer)는 컨볼루션이 아닌 주의(attentional) 계층을 사용하여 이 프레임워크를 그래프 영역으로 확장합니다. 또한, 본 논의에서는 Satorras 등(2021)의 특수 사례 솔루션에 초점을 맞추었지만, 그래프에 대한 회전 또는 이동 등변수 예측에 대한 동기는 역사적으로 포인트 클라우드용 동적 그래프 CNN(Dynamic Graph CNN )(Wang 등, 2019b)과 같은 아키텍처와 양자 화학을 위한 효율적인 메시지 전달 모델인 SchNet(Schütt 등, 2018) 및 DimeNet(Klicpera 등, 2020) 등 다른 분야에서 탐구되어 왔다는 점에 주목할 필요가 있습니다. 

### 일반 표현
Regular representations

환원 불가능한 표현 방식은 매력적이지만, 기본( underlying) 그룹 표현에 대해 직접 추론해야 하므로 지루할 수 있으며, 간결한(compact) 그룹에만 적용할 수 있습니다. 일반 표현 접근 방식은 더 일반적이지만, 정확한 동등성(equivariance)을 위해 모든 그룹 요소에 대한 잠재적 특징 임베딩의 복사본을 저장해야 한다는 추가적인 계산 부담이 있습니다. 

>사실 이 접근 방식은 이전 섹션에서 소개한 그룹 컨볼루션 신경망에 의해 개척된 것입니다. 

이 분야에서 유망한 접근법 중 하나는 지수(exponential) 및 로그(logarithmic) 맵의 정의를 통해 Lie 군(Lie groups)에 대한 동등성을 관찰하는 것으로, 다양한 대칭 군에 걸쳐 빠른 프로토타이핑(prototyping)을 약속합니다. Lie 그룹은 이 섹션의 범위를 벗어나지만, 최근 이 방향의 성공적인 사례로 Finzi 외(2020)의 LieConv와 Hutchinson 외(2020)의 LieTransformer를 소개합니다. 

이 섹션에서 다루는 접근법은 기하 그래프(geometric graphs)에서 기본 기하(underlying geometry)와 명시적(explicitly)으로 등변량(equivariant)인 방식으로 데이터를 처리하는 널리 사용되는 방법을 나타냅니다. 4.6절에서 설명한 것처럼, 메쉬(meshes)는 연속 곡면(continuous surfaces)의 이산(discretisations)으로 이해할 수 있는 기하 그래프의 특수한 인스턴스입니다. 다음에는 메시 전용 등변량(equivariant) 신경망에 대해 알아보겠습니다. 

## 5.6 고유 메시 CNN
Intrinsic Mesh CNNs

메쉬, 특히 삼각형 메쉬는 컴퓨터 그래픽의 '빵과 버터'이며 3D 객체를 모델링하는 가장 일반적인 방법입니다. 일반적으로 딥 러닝과 특히 컴퓨터 비전에서 CNN이 괄목할 만한 성공을 거두면서 2010년대 중반부터 그래픽 및 지오메트리 처리 커뮤니티에서 메시 데이터를 위한 유사한 아키텍처를 구축하는 데 큰 관심을 갖게 되었습니다.

### 측지 패치
Geodesic patches

메시에서 딥러닝을 위한 대부분의 아키텍처는 지수(exponential) 맵을 이산(discretising) 또는 근사화(approximating)하여 접하는 평면(tangent plane)의 좌표계로 필터를 표현함으로써 (21)과 같은 형태의 컨볼루션 필터를 구현합니다. 측지(geodesic) $γ : [0, T] → Ω$을 점 $u = γ(0)$에서 인근 점 $v = γ(T)$로 쏘면(Shooting) 측지 극좌표(geodesic polar coordinates)의 로컬 시스템 $(r(u, v), ϑ(u, v))$이 정의되며, 여기서 $r$은 $u$와 $v$ 사이의 측지 거리(측지 $γ$의 길이)이고 $ϑ$는 $γ'(0)$과 일부 로컬 기준 방향(local reference direction) 사이의 각도입니다. 이를 통해 측지 패치(geodesic patch) $x(u, r, ϑ) = x(\exp_u \tilde{ω}(r, ϑ))$를 정의할 수 있으며, 여기서 $\tilde{ω}_u : [0, R] × [0, 2π) → T_uΩ$은 로컬 극좌표 프레임입니다.

메쉬로 이산화된 표면에서, 측지선(geodesic)은 삼각형 면을 가로지르는 폴리라인입니다. 전통적으로 측지선은 매질에서의 파동 전파에 대한 물리적 모델에서 발생하는 아이코널 방정식(eikonal equation)이라고 하는 비선형 PDE(nonlinear PDE)의 효율적인 수치 근사치인 Kimmel과 Sethian(1998)의 고속 행군 알고리즘(Fast Marching algorithm)을 사용하여 계산해 왔습니다. 이 방식은 Kokkinos 등(2012)이 국부 측지 패치( local geodesic patches) 계산에 적용했으며, 이후 Masci 등(2015)이 메시에서 ,최초의 내재적 CNN과 유사한 아키텍처인, 측지(Geodesic) CNN을 구축하는 데 재사용했습니다. 

>측지 패치의 예시. 결과 패치가 위상적인 원반(topological disk)가 되려면 반경 $R$이 주입 반경(injectivity radius)보다 작아야 합니다.

>메시에서 이산 측지선(discrete geodesics)을 구성합니다.

### 등방성 필터
Isotropic filters

중요한 것은 측지 패치의 정의에서 기준 방향(reference direction)과 패치 방향(e patch orientation)의 선택에 모호성이 있다는 것입니다. 이것이 바로 게이지 선택의 모호성이며, 로컬 좌표계는 모든 노드에서 다를 수 있는 임의의 회전(또는 각도 좌표의 이동인 $x(u, r, ϑ + ϑ_0))$까지 정의됩니다. 가장 간단한 해결책은 $θ(r)$ 형태의 등방성(isotropic) 필터를 사용하여 이웃 특성(features)의 방향에 독립적인 집계를 수행하는 것입니다,

$$
    (x \star θ)(u) = \int^R_0 \int^{2π}_0 x(u, r, ϑ)θ(r)drdϑ
$$

4.4-4.6절에서 설명한 스펙트럼 필터는 등방성(isotropic)인 라플라시안 연산자를 기반으로 하는 이 범주에 속합니다. 그러나 이러한 접근 방식은 중요한 방향 정보를 버리고 가장자리와 같은 특징(edge-like features)을 추출하지 못할 수 있습니다. 

### 고정 게이지
Fixed gauge

4.4절에서 이미 언급한 바 있는 대안은 일부 게이지를 고정(fix some gauge)하는 것입니다. Monti 등(2017)은 주요 곡률 방향(principal curvature directions)을 사용했는데, 이 선택은 내재적이지 않고, 평평한 지점(곡률이 사라지는 곳)이나 균일한 곡률(완벽한 구 등)에서는 모호할 수 있지만, 저자는 이 방법이 대략 조각적으로 단단한(piecewise-rigid) 변형 가능한(deformable) 인체 모양을 처리하는 데 합리적이라는 것을 보여 주었습니다. 이후 Melzi 등(2019)의 연구에서는 고유 함수(intrinsic functions.)의 (고유) 기울기로 계산된 메시에서, 게이지의 신뢰할 수 있는 고유 구성(reliable intrinsic construction)을 보여주었습니다. 이러한 탄젠트 필드에는 특이점(즉, 특정 지점에서 사라짐)이 있을 수 있지만, 전반적인 절차는 노이즈와 재메싱에 매우 견고합니다.

### 앵귤러 풀링
Angular pooling

각도 최대 풀링(angular max pooling)이라고 하는 또 다른 접근 방식은 Masci 등(2015)에서 사용되었습니다. 이 경우 필터 $θ(r, ϑ)$는 이방성(anisotropic, 비등방성)이지만 함수와의 매칭은 가능한 모든 회전에 대해 수행된 다음 합산됩니다:

$$
    (x \star θ)(u) = \max_{ϑ_0∈[0,2π)} \int^R_0 \int^{2π}_0 x(u, r, ϑ)θ(r, ϑ + ϑ_0)drdϑ.
$$

개념적으로는 측지 패치를 회전 필터와 상호 연관시키고 가장 강력한 응답을 수집하는 것으로 시각화할 수 있습니다.

메시에서 연속 적분은 패치 연산자(patch operators)라고 하는 구조를 사용하여 이산화할 수 있습니다(Masci et al., 2015). 노드 $u$ 주변의 측지 패치에서 국부 극좌표( local polar coordinates)에서 $(r_{uv}, ϑ_{uv})$로 표시되는 이웃 노드 $\mathcal{N}_u$는 가중치 함수 세트 $w1(r, ϑ), ... . , w_K(r, ϑ)$(그림 18에 표시되어 있으며 '소프트 픽셀(soft pixels)' 역할을 함)로 가중치를 부여하고 합산(aggregated)됩니다,

$$
    (x \star θ)_u = \frac{\sum^K_{k=1} w_k \sum_{v∈\mathcal{N}_u} (r_{uv}, ϑ_{uv})x_v θ_k}{\sum^K_{k=1} w_k \sum_{v∈\mathcal{N}_u} (r_{uv}, ϑ_{uv}) θ_k}
$$

(여기서 $θ_1, . . . , θ_K$는 필터의 학습 가능한 계수입니다). 다중 채널 특징(features)은 적절한 필터 제품군을 사용하여 채널별로 처리됩니다. Masci 등(2015), Boscaini 등(2016a)은 사전 정의된 가중치 함수 $w$를 사용했고, Monti 등(2017)은 이를 학습할 수 있도록 허용했습니다. 

>일반적으로 멀티홉(multi-hop) 이웃이 사용됩니다.

### 게이지 등변량 필터 
Gauge-equivariant filters

등방성(isotropic) 필터와 각도 최대 풀링(angular max pooling)은 모두 게이지 변환에 불변(invariant)하는 특징을 유도하며, 이들은 사소한 표현(trivial representation)$ρ(\mathfrak{g}) = 1$(여기서 $\mathfrak{g} ∈ SO(2)$는 로컬 좌표 프레임의 회전)에 따라 변환됩니다. 이러한 관점은 네트워크에 의해 계산된 특징이 구조 그룹 $\mathfrak{G}$의 (예: 각각 좌표 프레임의 회전(rotations)의 $SO(2)$ 또는 회전+반사(rotations+reflections)의 $O(2)$) 임의의 표현 $ρ$와 연관되는 Cohen 등(2019), de Haan 등(2020)이 제안하고 4.5장에서 설명한 또 다른 접근 방식을 제안합니다. 탄젠트 벡터는 표준 표현 $ρ(\mathfrak{g}) = g$에 따라 변환합니다. 또 다른 예로, 동일한 필터의 회전된 복사본 n개를 일치시켜 얻은 특징 벡터는, 게이지의 회전에 따라 주기적 이동(cyclic shifts)에 의해 변환(transforms)되며, 이는 주기 그룹(cyclic group) $C_n$의 정규 표현으로 알려져 있습니다.

4.5절에서 설명한 것처럼, (사소하지 않은(non-trivial) 표현과 관련된) 이러한 기하학적 특징을 다룰 때는, 필터를 적용하기 전에 먼저 동일한 벡터 공간으로 병렬 전송(parallel transport)해야 합니다. 메시에서 이 작업은 de Haan 등(2020)이 설명한 다음과 같은 메시지 전달 메커니즘을 통해 구현할 수 있습니다. $\mathbf{x}_u ∈ \mathbb{R}^d$을 메쉬 노드 $u$에서 $d$ 차원 입력 특징(feature)이라고 합시다. 이 특징(feature)은 (임의의) 선택된 게이지($u$)를 기준으로 표현되며, 게이지의 회전 시 $\mathfrak{G} = SO(2)$의 표현 $ρ_{in}$에 따라 변환된다고 가정합니다. 마찬가지로 메시 컨볼루션의 출력 특징(output features) $\mathbf{h}_u$는 $d'$ 차원이며 $ρ_{out}$에 따라 변환되어야 합니다(네트워크 설계자가 임의로 선택할 수 있음). 

그래프 신경망(GNN)과 유사하게, 메시에서 $u$의 이웃 $\mathcal{N}_u$(및 $u$ 자체)에서 $u$로 메시지를 전송하여 게이지 등변량(gauge-equivariant) 컨볼루션(23)을 구현할 수 있습니다:

$$
    \mathbf{h}_u = Θ_{\text{self}} \mathbf{x}_u + \sum_{v∈\mathcal{N}_u} Θ_{\text{neigh}}(ϑ_{uv})ρ(\mathfrak{g}_{v→u})\mathbf{x}_v, \tag{37}
$$

여기서 $Θ_{\text{self}}, Θ_{\text{neigh}} (ϑ_{uv}) ∈ \mathbb{R}^{d'×d}$는 학습된 필터 행렬입니다. 구조 그룹 요소 $\mathfrak{g}_{v→u} ∈ SO(2)$는 $v$에서 $u$로의 병렬 전송 효과를 나타내며, $u$와 $v$의 게이지를 기준으로 표현되며, 각 메시마다 미리 계산할 수 있습니다. 그 작용은 트랜스포터 행렬(transporter matrix) $ρ(\mathfrak{g}_{v→u}) ∈ \mathbb{R}^{d×d}$ 로 인코딩됩니다. 행렬 $Θ_{\text{neigh}}(ϑ_{uv})$는 $u$에서 기준 방향(예: 프레임의 첫 번째 축)에 대한 이웃 $v$의 각도 $ϑ_{uv}$에 따라 달라지므로, 이 커널은 이방성(anisotropic)이므로 서로 다른 이웃은 다르게 처리됩니다.

>$d$는 피처 차원이며 반드시 메시의 차원인 2와 같을 필요는 없습니다.

4.5절에서 설명한 것처럼, $\mathbf{h}(u)$가 잘 정의된 기하학적 양(quantity)이 되려면 게이지 변환 하에서 $\mathbf{h}(u) \mapsto ρ_{out}(\mathfrak{g}^{-1} (u))\mathbf{h}(u)$로 변환되어야 합니다. 이는 모든 $ϑ ∈ SO(2)$에 대해 $Θ_{\text{self}ρ_{in}}(ϑ) = ρ_{out}(ϑ)Θ_{\text{self}}$이고 $Θ_\text{neigh}(ϑ_{uv} - ϑ)ρ_{in}(ϑ) = ρ_{out}(ϑ)Θ_{\text{neigh}}(ϑ_{uv})$인 경우에 해당됩니다. 이러한 제약 조건은 선형이므로, 이러한 제약 조건을 만족하는 행렬 $Θ_{\text{self}}$와 행렬값 함수 $Θ_{\text{neigh}}$의 공간은 그런 조건들을 만족시켜 선형 하위공간입니다. 이에 학습 가능한 계수가 있는 기저 커널(basis kernels)의 선형 조합으로 파라미터화할 수 있습니다: $Θ_{\text{self}} = \sum_i α_iΘ^i_{\text{self}}$ 및 $Θ_{\text{neigh}} = \sum_i β_iΘ^i_{\text{neighbor}}$.

>여기서는 표기법을 남용(abuse)하여 각도 $ϑ$로 2D 회전을 식별합니다.


## 5.7 반복 신경망
Recurrent Neural Networks

지금까지의 논의에서는 항상 입력이 주어진 도메인에 걸쳐 공간적(spatial)으로만 존재한다고 가정했습니다. 그러나 많은 일반적인 사용 사례에서는 입력이 순차적(sequential)이라고 간주할 수도 있습니다(예: 비디오, 텍스트 또는 음성). 이 경우 입력이 임의로 많은 단계로 구성되며, 각 단계 $t$마다 입력 신호가 제공되며, 이를 $\mathbf{X}^{(t)} ∈ \mathcal{X} (Ω^{(t)} )$로 표현한다고 가정합니다. 

>도메인을 정적(static)인 것으로 간주할지 동적(dynamic)인 것으로 간주할지는 시간 척도(time scales)와 관련이 있습니다. 예를 들어, 도로망은 시간이 지남에 따라 (새로운 도로가 건설되고 오래된 도로가 철거됨에 따라) 변화하지만 트래픽의 흐름에 비해 훨씬 느리게 변화합니다. 마찬가지로 소셜 네트워크에서 참여도(engagement)의 변화(예: 트위터 사용자가 트윗을 리트윗하는 것)는 팔로우 그래프의 변화보다 훨씬 더 높은 빈도로 일어납니다.

일반적으로 도메인은 신호와 함께 시간에 따라 진화할 수 있지만, 일반적으로 도메인은 모든 $t$에 걸쳐 고정된 상태로 유지된다고 가정합니다(즉, $Ω^{(t)} = Ω$). 여기서는 이 경우에만 초점을 맞출 것이지만, 예외도 흔하다는 점에 유의하세요. 소셜 네트워크는 새로운 링크가 정기적으로 생성되고 삭제되기 때문에 시간이 지남에 따라 도메인이 변경되는 경우를 고려해야 하는 예입니다. 이 설정에서 도메인은 종종 동적 그래프(dynamic graph)라고 불립니다(Xu et al., 2020a; Rossi et al., 2020). 

종종 개별 $\mathbf{X}^{(t)}$ 입력은 유용한 대칭을 나타내므로 앞서 설명한 아키텍처에서 중요하지 않게(nontrivially) 처리될 수 있습니다. 몇 가지 일반적인 예로는 동영상($Ω$은 고정된 격자이고 신호는 프레임의 시퀀스), fMRI 스캔($Ω$은 뇌 피질의 기하학적 구조를 나타내는 고정된 메시로, 제시된 자극(stimuli)에 대한 반응으로 서로 다른 영역이 서로 다른 시간에 활성화됨), 교통 흐름 네트워크(Ω은 도로 네트워크를 나타내는 고정 그래프로, 다양한 노드에서 평균 교통 속도가 기록됨) 등을 들 수 있습니다.

입력 도메인의 대칭을 존중하면서 문제에 적합한 세분화(granularity) 수준에서 잠재적 표현을 제공하는 인코더 함수 $f(\mathbf{X}^{(t)})$가 있다고 가정해 보겠습니다. 예를 들어 비디오 프레임을 처리하는 경우를 생각해 보겠습니다. 즉, 각 타임스텝마다 $n×d$ 행렬 $\mathbf{X}^{(t)}$로 표현되는 그리드 구조의 입력(grid-structured input)이 주어지며, 여기서 $n$은 픽셀 수(시간 고정)이고 $d$는 입력 채널 수입니다(예: RGB 프레임의 경우 $d = 3$). 또한 전체 프레임 수준에서의 분석에 관심이 있으며, 이 경우 $f$를 변환 불변(translation invariant) CNN으로 구현하여, 시간 단계 t에서 프레임의 $k$ 차원 표현 $\mathbf{z}^{(t)} = f(\mathbf{X}^{(t)})$를 출력하는 것이 적절합니다.

>이 예제에서 일반성을 잃지 않습니다; 시공간(spatiotemporal) 그래프의 노드 수준 출력에 대해 동등한(equivalent) 분석을 수행할 수 있습니다. 유일한 차이점은 인코더 $f$(이 경우 순열 등변량(permutation equivariant) GNN이 됨)의 선택에 있습니다.

이제 모든 단계에 걸쳐 일련의 벡터 $z^{(t)}$를 적절하게 요약(summarising)하는 작업이 남았습니다. 입력의 시간적 진행을 존중하고 새로운 데이터 포인트의 온라인 도착(online arrival)을 쉽게 허용하는 방식으로 이 정보를 동적(dynamically)으로 집계하는 일반적인(canonical) 방법은 순환 신경망(Recurrent Neural Network)(RNN)을 사용하는 것입니다. 여기서 보여드리는 것은 RNN이 입력 $\mathbf{z}^{(t)}$에 대해 다소 특이한 유형의 대칭을 구현하기 때문에 그 자체로 연구하기에 흥미로운 기하학적 아키텍처라는 것입니다.

>$z^{(t)}$ 벡터는 시간적 격자(temporal grid)의 점으로 볼 수 있으므로 경우에 따라 CNN으로 처리하는 것도 가능합니다. 트랜스포머는 일반적인 순차 입력을 처리하는 데 점점 더 많이 사용되는 모델이기도 합니다.

### 단순한 RNN
SimpleRNN

각 단계에서 순환 신경망은 $t$까지의 모든 입력 단계에 대한 $m$차원 요약 벡터 $\mathbf{h}^{(t)}$를 계산합니다. 이 (부분) 요약은 다음과 같이 공유 업데이트 함수$(\mathcal{R} : \mathbb{R}^k × \mathbb{R}^m → \mathbb{R}^m)$를 통해 현재 단계의 특징과 이전 단계의 요약에 따라 조건부로 계산됩니다(요약은 그림 19 참조): 

$$
   \mathbf{h}^{(t)} = \mathcal{R} (\mathbf{z}^{(t)}, \mathbf{h}^{(t−1)})    \tag{38}
$$

그리고 $\mathbf{z}^{(t)}$와 $\mathbf{h}^{(t-1)}$는 모두 평평한(flat) 벡터 표현이므로, $\mathcal{R}$은 완전히 연결된 단일 신경망 계층(single fully-connected neural network layer)으로 가장 쉽게 표현할 수 있습니다(종종 SimpleRNN이라고도 함, Elman(1990), Jordan(1997) 참조): 

$$
    \mathbf{h}^{(t)} = σ(\mathbf{Wz}^{(t)} + \mathbf{Uh}^{(t−1)} + \mathbf{b}) \tag{39}
$$

여기서 $\mathbf{W} ∈ \mathbb{R}^{k×m}, \mathbf{U} ∈ \mathbb{R}^{m×m}, \mathbf{b} ∈ \mathbb{R}^m$은 학습 가능한 파라미터이고, $σ$는 활성화 함수입니다. 이렇게 하면 네트워크의 계산 그래프에 루프(loops)가 발생하지만, 실제로는 네트워크가 적절한 단계 수만큼 풀리므로 시간에 따른 역전파(backpropagation through time)(Robinson and Fallside, 1987; Werbos, 1988; Mozer, 1989)가 적용될 수 있습니다.

>이름과는 달리 SimpleRNN은 표현력이 매우 뛰어납니다. 예를 들어, Siegelmann과 Sontag(1995)는 이러한 모델이 튜링 완전형(Turing-complete)이며, 이는 우리가 컴퓨터에서 실행할 수 있는 모든 연산을 표현할 수 있다는 것을 의미합니다.

그런 다음 요약 벡터를 다운스트림 작업에 적절히 활용할 수 있습니다. 시퀀스의 모든 단계에서 예측이 필요한 경우, 공유 예측자(shared predictor)를 각 $\mathbf{h}^{(t)}$에 개별적으로 적용할 수 있습니다. 전체 시퀀스를 분류할 때는 일반적으로 최종 요약인 $\mathbf{h}^{(T)}$가 분류기로 전달됩니다. 여기서 $T$는 시퀀스의 길이입니다. 

특히 초기 요약 벡터는 일반적으로 제로 벡터, 즉 $\mathbf{h}^{(0)} = \mathbf{0}$으로 설정되거나, 학습이 가능하도록 만들어집니다. 초기 요약 벡터가 설정되는 방식을 분석하면 RNN이 보여주는 흥미로운 형태의 번역 등식(translation equivariance)을 추론할 수 있습니다.

### RNN의 번역 등가성
Translation equivariance in RNNs

개별 단계 $t$를 불연속적인 시간 단계(discrete time-steps)로 해석하기 때문에 입력 벡터 $\mathbf{z}^{(t)}$는 1차원 시간 단계 그리드에 존재하는 것으로 볼 수 있습니다. 여기서 CNN의 번역 등차 분석(translation equivariance analysis)을 확장하는 것이 매력적일 수 있지만, 간단한 방법으로 할 수는 없습니다.

>이 구조는 그리드를 더 높은 차원으로 확장할 수 있습니다. 예를 들어 이미지에 존재하는 신호를 스캔라인(scanline) 방식으로 처리할 수 있습니다. 이러한 구조는 van den Oord 외(2016b)의 PixelRNN과 같은 인기 있는 일련의 모델을 구동했습니다.

그 이유를 알아보기 위해 수열을 한 단계씩 좌변환하여 새로운 수열 $\mathbf{z}'^{(t)} = \mathbf{z}^{(t+1)}$을 생성했다고 가정해 보겠습니다. 변환 동등성(translation equivariance)에서 기대할 수 있는 것처럼 $\mathbf{h}'^{(t)} = \mathbf{h}^{(t+1)}$을 표시하려고 시도하고 싶을 수 있지만, 일반적으로는 그렇지 않습니다. $t = 1$이라고 가정하고 업데이트 함수를 직접 적용하고 확장하면 다음과 같은 결과를 얻을 수 있습니다: 

$$
\begin{align}
    \mathbf{h'}^{(t)} & = \mathcal{R} (\mathbf{z'}^{(1)}, \mathbf{h}^{(0)}) 
    = \mathcal{R} (\mathbf{z}^{(2)}, \mathbf{h}^{(0)})  
\tag{40}
\\
    \mathbf{h}^{(2)} & = \mathcal{R} (\mathbf{z}^{(2)}, \mathbf{h}^{(1)}) 
    = \mathcal{R} (\mathbf{z}^{(2)}, (\mathbf{z}^{(1)}, \mathbf{h}^{(0)}))  
\tag{41}
\end{align}
$$

따라서 $\mathbf{h}^{(0)} = \mathcal{R} (\mathbf{z}^{(1)}, \mathbf{h}^{(0)})$을 보장할 수 없다면 변환 등식(translation equivariance)을 복구하지 못합니다. $t > 1$ 단계에 대해 유사한 분석을 수행할 수 있습니다.

다행히도 $\mathbf{z}$를 표현하는 방법을 약간만 리팩토링하고, $\mathcal{R}$을 적절히 선택하면 위의 등식을 만족시킬 수 있으며, 따라서 RNN이 시프트와 등변량(equivariant to shifts)인 설정을 보여줄 수 있습니다. 우리의 문제는 주로 경계 조건(boundary conditions) 중 하나였습니다. 경계 조건: 위의 방정식에는 좌변환 연산(leftshift operation )으로 파괴된 $\mathbf{z}^{(1)}$이 포함되어 있습니다. 이 문제를 추상화하기 위해, 다음과 같이 정의된 왼쪽으로 덧셈된(left-padded) 시퀀스 $\bar{\mathbf{z}}^{(t)}$를 RNN이 어떻게 처리하는지 관찰해 보겠습니다: 

$$
    \bar{\mathbf{z}}^{(t)} = 
    \begin{cases}        
                \mathbf{0} & t ≤ t' 
                \\
                \mathbf{z}^{(t-t')} & t > t'
    \end{cases}
$$


이러한 시퀀스는 이제 원래 입력 요소를 파괴하지 않고 최대 $t'$ 단계까지 왼쪽으로 이동할 수 있습니다. 또한, 오른쪽 시프트를 별도로 처리할 필요가 없으며, 실제로 오른쪽 시프트에 대한 등식(equivariance)은 RNN 방정식으로부터 자연스럽게 따라온다는 점에 유의하세요.

>0이 아닌 다른 패딩 벡터를 사용하면 동등한 분석(equivalent analyses)이 발생할 수 있다는 점에 유의하세요.

이제 방정식 40-41에서와 마찬가지로 $\bar{\mathbf{z}}'^{(t)} = \bar{\mathbf{z}}^{(t=1)}$로 표시되는 왼쪽 시프트된 버슨에 대한 RNN의 연산을 다시 분석할 수 있습니다: 

$$
\begin{align}
    \mathbf{h'}^{(1)} & = \mathcal{R} (\bar{\mathbf{z}}'^{(1)}, \mathbf{h}^{(0)}) 
     = \mathcal{R} (\bar{\mathbf{z}}^{(2)}, \mathbf{h}^{(0)})   
\\
    \mathbf{h}^{(2)} & = \mathcal{R} (\bar{\mathbf{z}}^{(2)}, \mathbf{h}^{(1)}) 
    = \mathcal{R} (\bar{\mathbf{z}}^{(2)}, \mathcal{R} (\bar{\mathbf{z}}^{(1)}, \mathbf{h}^{(0)})) = \mathcal{R} (\bar{\mathbf{z}}^{(2)}, \mathcal{R} (\mathbf{0}, \mathbf{h}^{(0)}))
\end{align}
$$

여기서 치환 $\bar{\mathbf{z}}^{(1)} = 0$은 $t' ≥ 1$인경우, 즉 어떤 패딩이 적용되는 한, 유지됩니다. 이제 $\mathbf{h}^{(0)} = \mathcal{R} (\mathbf{0}, \mathbf{h}^{(0)})$이면 한 단계 좌변환$(\mathbf{h'}^{(t)} = \mathbf{h'}^{(t+1)})$에 대한 등가성(equivariance)을 보장할 수 있습니다. 

>매우 유사한 맥락에서, $t'≥ s$인 경우 좌측 이동에 대한 등식(equivariance)을 s 단계씩 도출할 수 있습니다.

다르게 말하면, $\mathbf{h}^{(0)}$는 $γ(\mathbf{h}) = \mathcal{R} (\mathbf{0}, \mathbf{h})$ 함수의 고정점(fixed point)으로 선택되어야 합니다. 업데이트 함수 $\mathcal{R}$을 편리하게 선택하면, 이러한 고정점의 존재를 보장할 수 있을 뿐만 아니라, 다음과 같이 수렴할 때까지 $\mathcal{R}$ 을 반복적으로 적용하여 고정점을 직접 구할 수도 있습니다:

$$
     \mathbf{h}_0 = \mathbf{0} \quad  \mathbf{h}_{k+1} = γ(\mathbf{h}_{k})  \tag{42}
$$

여기서 인덱스 $k$는, RNN의 시간 단계를 나타내는 인덱스$(t)$와는 반대로, 계산에서 $\mathcal{R}$의 반복을 나타냅니다. $γ$가 수축 매핑(contraction mapping)이 되도록 $\mathcal{R}$을 선택하면 이러한 반복은 실제로 고유한(unique) 고정점에 수렴합니다. 따라서 방정식 (42)를 $\mathbf{h}_{k+1} = \mathbf{h}_k$가 될 때까지 반복하면 $\mathbf{h}^{(0)} = \mathbf{h}_k$로 설정할 수 있습니다. 이 계산은 수열에 "충분히 많은(sufficiently many)" 영벡터를 왼쪽으로 덧셈하는 것과 동일합니다.

>수축(Contractions)은 $γ : \mathcal{X} → \mathcal{X}$ 함수로, $\mathcal{X}$에 대한 어떤 규범(norm) $\| \cdot \|$ 아래에서, $γ$를 적용하면 점 사이의 거리가 수축(contracts)합니다: 모든 $x, y ∈ \mathcal{X}$ , 그리고 어떤 $q ∈ [0, 1)$에서, $\|γ(x) - γ(y)\| ≤ q\|x - y\|$를 유지합니다. 이러한 함수를 반복하면 바나흐의 고정점 정리(Banach’s Fixed Point Theorem)(Banach, 1922)의 직접적인 결과로서 반드시 고유한(unique) 고정점에 수렴합니다.

### RNN의 깊이
Depth in RNNs

또한 여러 개의 RNN을 쉽게 스택할 수 있습니다. $\mathbf{h}^{(t)}$ 벡터를 두 번째 RNN의 입력 시퀀스로 사용하기만 하면 됩니다. 이러한 종류의 구조를 때때로 'deep RNN'이라고 부르기도 하는데, 이는 오해의 소지가 있습니다. 실제로 반복 연산을 반복적으로 적용하기 때문에 단일 RNN '레이어'도 입력 단계 수와 동일한 깊이를 갖습니다. 

각 훈련 예제가 업데이트 네트워크의 공유 매개변수(shared parameters)에 대한 많은 경사 업데이트를 유도하기 때문에, RNN을 최적화할 때 종종 고유하게 까다로운 학습 역학이 발생합니다. 여기서는 이러한 문제 중 가장 두드러진 문제인 소실(vanishing) 및 폭발(exploding) 그래디언트(Bengio et al., 1994)에 초점을 맞출 것인데, 이는 깊이와 매개변수 공유를 고려할 때 RNN에서 특히 문제가 되는 문제입니다. 또한, 이 연구는 RNN에 대한 가장 영향력 있는 연구 중 일부에 박차(spurred)를 가했습니다. 보다 자세한 개요를 보려면 RNN의 훈련 역학을 매우 상세히 연구하고 분석적, 기하학적, 동적 시스템의 렌즈 등 다양한 관점에서 이러한 문제를 드러낸 파스카누 외(Pascanu et al)(2013)의 논문을 참조하시기 바랍니다.

소실(vanishing) 그라데이션을 설명하기 위해 미분 크기 $|σ'|$가 항상 $0$과 $1$ 사이인 시그모이드(sigmoidal) 활성화 함수 σ를 가진 SimpleRNN을 생각해 보겠습니다. 이러한 값을 많이 곱하면 그라데이션이 빠르게 0이 되는 경향이 있으며, 이는 입력 시퀀스의 초기 단계가 네트워크 파라미터 업데이트에 전혀 영향을 미치지 못할 수 있다는 것을 의미합니다.

>이러한 활성화의 예로는 로지스틱 함수(logistic function)인 $σ(x) = \frac{1}{1+\exp(-x)}$ 와 쌍곡선 접선인 $σ(x) = \tanh x$가 있으며, 플롯의 뚜렷한 S자 모양으로 인해 시그모이달(sigmoidal)이라고 불립니다.

예를 들어, 다음 단어 예측 작업(예: 예측 키보드에서 일반적)과 입력 텍스트 "Petar는 세르비아인입니다. 그는 ...에서 태어났습니다. [긴 단락] . . . 페타르는 현재 ___ " 에 살고 있습니다. 여기서 다음 단어를 "세르비아"로 예측하는 것은 단락의 시작 부분만 고려하면 합리적으로 결론을 내릴 수 있지만, 이 입력 단계에 도달하면 그라데이션이 사라질 가능성이 높기 때문에 이러한 예제에서 학습하기가 매우 어렵습니다. 

딥 피드포워드 신경망(Deep feedforward neural networks)도 소실 경사 문제( vanishing gradient problem)로, ReLU 활성화(경사도가 정확히 0 또는 1과 같아서 소실 경사 문제를 해결)가 발명되기 전까지 고통받았습니다. 그러나 이제 업데이트 함수의 출력 공간이 제한(unbounded)되지 않고, 기울기 하강이 모든 입력 단계마다 셀을 한 번씩 업데이트하여 업데이트 규모가 빠르게 증가하기 때문에 RNN에서 ReLU를 사용하면 기울기가 폭발(exploding)적으로 증가하기 쉽습니다. 역사적으로 소실 그라디언트 현상은 순환 네트워크를 사용하는 데 있어 중요한 장애물로 일찍부터 인식되어 왔습니다. 이 문제를 해결하기 위해 보다 정교한(sophisticated) RNN 레이어를 개발하게 되었고, 이에 대해 다음에서 설명합니다.

## 5.8 장단기 메모리 네트워크
Long Short-Term Memory networks

RNN에서 소실 그라데이션의 영향을 크게 줄인 핵심 발명품은 네트워크가 데이터 기반 방식으로 정보를 선택적으로 덮어쓸 수 있도록(overwrite) 하는 게이팅 메커니즘(gating mechanisms)입니다. 이러한 게이트 RNN의 대표적인 예로는 장단기 메모리(Long Short-Term Memory)(LSTM; Hochreiter and Schmidhuber (1997))와 게이트 순환 유닛(Gated Recurrent Unit)(GRU; Cho et al. (2014))이 있습니다. 여기서는 이러한 모델의 작동을 설명하기 위해 주로 LSTM, 특히 Graves(2013)가 제시한 변형에 대해 논의할 것입니다. LSTM의 개념은 다른 게이트 RNN으로 쉽게 넘어갈 수 있습니다.

이 섹션에서는 본문에서 설명할 모든 LSTM 연산을 설명하는 그림 20을 참조하는 것이 유용할 것입니다. 

LSTM은 계산 단계 사이에 보존(preserved)되는 셀 상태(cell state) 벡터 $\mathbf{c}^{(t)} ∈ \mathbb{R}^m$ 을 저장하는 메모리 셀(memory cell)을 도입하여 반복 계산을 보강(augments)합니다. LSTM은 $\mathbf{c}^{(t)}$ 를 기반으로 직접 요약 벡터인 $\mathbf{h}^{(t)}$ 를 계산하고, $\mathbf{c}^{(t)}$ 는 차례로 $\mathbf{z}^{(t)}, \mathbf{h}^{(t-1)}$ 및 $\mathbf{c}^{(t-1)}$ 를 사용하여 계산됩니다. 중요한 점은, 셀이 $\mathbf{z}^{(t)}$ 및 $\mathbf{h}^{(t-1)}$ 을 기반으로 완전히 덮어쓰지 않기 때문에, 네트워크가 SimpleRNN과 동일한 문제에 노출될 수 있다는 점입니다. 대신, 이전 셀 상태의 일정량이 유지(retained)될 수 있으며, 이러한 비율이 발생하는 비율은 데이터에서 명시적으로 학습됩니다.

SimpleRNN에서와 마찬가지로, 현재 입력 단계와 이전 요약에 대해 완전히 연결된 단일 신경망 레이어(single fully-connected neural network layer)를 사용하여 특징을 계산합니다: 

$$
\begin{align}
    \tilde{c}^{(t)} = \tanh (\mathbf{W}_c\mathbf{z}^{(t)} + \mathbf{U}_c \mathbf{h}^{(t-1)} + \mathbf{b}_c)
    \tag{43} \\
\end{align}
$$
.

>여기서는 활성화 함수를 $\tanh$ 로 설정했는데, LSTM은 소실 그라데이션 문제를 개선(ameliorate)하도록 설계되었으므로 이제 시그모이드(sigmoidal) 활성화를 사용하는 것이 적절합니다.

그러나 앞서 언급했듯이 이 벡터가 모두 셀에 들어가는 것은 허용되지 않으므로, 이를 후보 특징 벡터(the vector of candidate features)라고 부르고 $\tilde{c}^{(t)}$ 로 표시합니다. 대신, LSTM은 $[0, 1]$ 범위의 실수값 벡터인 게이팅 벡터(gating vectors)를 직접 학습하여 메모리 셀에 얼마나 많은 신호가 들어오고, 나가고, 덮어쓰도록 허용할지 결정합니다.

셀에 들어갈 수 있는 후보 벡터의 비율을 계산하는 입력 게이트(input gate) $\mathbf{i}^{(t)}$ , 유지될 이전 셀 상태의 비율을 계산하는 잊기 게이트(forget gate) $\mathbf{f}^{(t)}$ , 최종 요약 벡터에 사용될 새로운 셀 상태의 비율을 계산하는 출력 게이트(output gat) $\mathbf{o}^{(t)}$ 의 세 가지 게이트가 모두 $\mathbf{z}^{(t)}$ 와 $\mathbf{h}^{(t-1)}$ 에 기반하여 계산됩니다. 일반적으로 이러한 모든 게이트는 출력이 $[0, 1]$ 범위에 있도록 하기 위해 로지스틱 시그모이드(logistic sigmoid) 활성화 로지스틱 $(x) = \frac{1}{1+\exp(-x)}$ 를 사용하지만 완전히 연결된 단일 계층을 사용하여 파생됩니다: 

$$
\begin{align}
    \mathbf{i}^{(t)} & = \text{logistic} (\mathbf{W}_\text{i}\mathbf{z}^{(t)} + \mathbf{U}_\text{i} \mathbf{h}^{(t-1)} + \mathbf{b}_\text{i})
\tag{44} \\
    \mathbf{f}^{(t)} & = \text{logistic} (\mathbf{W}_\text{f}\mathbf{z}^{(t)} + \mathbf{U}_\text{f} \mathbf{h}^{(t-1)} + \mathbf{b}_\text{f}) 
\tag{45} \\
    \mathbf{o}^{(t)} & = \text{logistic} (\mathbf{W}_\text{o}\mathbf{z}^{(t)} + \mathbf{U}_\text{o} \mathbf{h}^{(t-1)} + \mathbf{b}_\text{o})
\tag{46}
\end{align}
$$
.

>세 개의 게이트는 그 자체로 벡터, 즉 $i^{(t)}, f^{(t)}, o^{(t)} ∈ [0, 1]^m$ 입니다. 이를 통해 게이트를 통과할 수 있는 각 차수 $m$ 의 양을 제어할 수 있습니다.

마지막으로, 이러한 게이트가 적절히 적용되어 새로운 셀 상태인 $\mathbf{c}^{(t)}$ 를 디코딩한 다음, 출력 게이트에 의해 변조되어 다음과 같이 요약 벡터 $\mathbf{h}^{(t)}$ 를 생성합니다: 

$$
\begin{align}
    \mathbf{c}^{(t)} & = \mathbf{i}^{(t)} \odot \tilde{\mathbf{c}}^{(t)} + \mathbf{f}^{(t)} \odot \mathbf{c}^{(t-1)}\tag{47} \\
    \mathbf{h}^{(t)} & = \mathbf{o}^{(t)} \odot \tanh(\mathbf{c}^{(t)})
\tag{48}
\end{align}
$$

$\odot$ 는 요소별 벡터 곱셈입니다. 방정식 (43)-(48)을 함께 적용하면 이제 셀 벡터 $\mathbf{c}^{(t)}$ 도 고려하는 LSTM의 업데이트 규칙(update rule)이 완전히 지정됩니다:

$$
\begin{equation}
(\mathbf{h}^{(t)}, \mathbf{c}^{(t)}) = \mathcal{R} (\mathbf{z}^{(t)}, (\mathbf{h}^{(t-1)}, \mathbf{c}^{(t-1)}))
\end{equation}
$$
.

>이는 방정식 (38)의 RNN 업데이트 청사진과도 여전히 호환되며, 요약 벡터를 $\mathbf{h}^{(t)}$ 와 $\mathbf{c}^{(t)}$ 의 연결(concatenation)로 간주하기만 하면 됩니다 (때로는 $\mathbf{h}^{(t)} \| \mathbf{c}^{(t)}$ 로 표시하기도 함). 

$\mathbf{f}^{(t)}$의 값은 $\mathbf{z}^{(t)}$ 와 $\mathbf{h}^{(t-1)}$ 에서 파생되므로, 데이터에서 직접 학습할 수 있습니다. LSTM은 과거 경험을 적절히 잊어버리는 방법을 효과적으로 학습할 수 있습니다. 실제로 $\mathbf{f}^{(t)}$의 값은 모든 LSTM 파라미터$(\mathbf{W_∗, U_∗, b_∗})$에 대한 역전파 업데이트에 직접 나타나므로 네트워크가 데이터 기반 방식으로 시간 단계에 걸친 기울기의 소실 정도를 명시적으로 제어(control)할 수 있습니다. 

소실 그라디언트 문제를 정면으로 해결한 것 외에도, 게이트형 RNN은 타임워핑(time-warping) 변환에 매우 유용한 형태의 불변성(invariance)을 제공하는 것으로 나타났는데, 이는 SimpleRNN이 도달할 수 없는 영역에 속합니다.

### 게이트 RNN의 시간 왜곡 불변성
Time warping invariance of gated RNNs

먼저 연속 시간(continuous-time) 환경에서 시간을 왜곡(warp time)한다는 것이 무엇을 의미하는지, 그리고 그러한 변환에 대한 불변성(invariance)을 달성하기 위해 순환 모델에 필요한 것이 무엇인지 설명하겠습니다. 이 설명은 이 현상을 처음 설명한 탈렉과 올리비에(Tallec and Ollivier)(2018)의 연구를 주로 따를 것이며, 실제로 이들은 불변성(invariances)의 관점에서 RNN을 실제로 연구한 최초의 연구자들 중 한 명입니다.

>연속 설정에 집중하는 이유는 시간 조작에 대한 추론이 더 쉬워지기 때문입니다. 

RNN을 적용하고자 하는 연속 시간 영역 신호 z(t)를 가정해 보겠습니다. 요약 벡터 $\mathbf{h}^{(t)}$에 대한 RNN의 불연속 시간(discrete-time) 계산을 연속 영역(continuous domain)의 아날로그 신호인 $h^{(t)}$와 일치시키기 위해 다음과 같은 선형 테일러 확장(linear Taylor expansion)을 관찰할 것입니다:

$$
    h(t + δ) ≈ h(t) + δ\frac{dh(t)}{dt}
\tag{49}
$$

$δ = 1$로 설정하면 $h(t)$와 $h(t+ 1)$ 사이의 관계를 복구할 수 있으며, 이는 정확히 RNN 업데이트 함수 R(방정식 38)이 계산하는 것과 같습니다. 즉, RNN 업데이트 함수는 다음 미분 방정식을 만족합니다: 

$$
    \frac{dh(t)}{dt} = h(t + 1) − h(t) = \mathcal{R}(z(t + 1), h(t)) − h(t)
\tag{50}
$$
.

>시간 $t$의 연속 신호를 나타내기 위해 $h(t)$를 사용하고 시간 단계 $t$의 이산 신호를 나타내기 위해 $\mathbf{h}^{(t)}$를 사용합니다.

우리는 신호의 불완전성이나 불규칙성을 설명하기 위해, 신호가 샘플링되는 방식(예: 측정 시간 단위 변경)에 대해 RNN이 탄력성(resilient)을 갖기를 원합니다. 공식적으로는 시간 워핑(time warping) 연산 $τ : \mathbb{R}^+ → \mathbb{R}^+$를 시간 간에 단조롭게(monotonically) 증가하는 미분 매핑으로 나타냅니다. τ라는 표기는 타임 워핑이 시간의 자동변형(automorphism)을 나타내기 때문에 선택된 표기법입니다.

>이러한 워핑 연산은 시간 리스케일링(time rescaling)과 같이 간단(simple)할 수 있습니다(예: τ(t) = 0.7t(위 표시), 이산 설정에서는 ∼ 1.43스텝마다 새로운 입력을 수신하는 것과 같습니다). 그러나, 샘플링이 시간 영역 전체에서 자유롭게 가속 또는 감속될 수 있는 등 가변적으로 변화하는(variably-changing) 샘플링 속도의 넓은 스펙트럼도 허용합니다.

또한, 클래스의 모든 모델과 그러한 $τ$ 에 대해, 워프되지 않은 경우 원본 모델과 동일한 방식으로 워프된 데이터를 처리하는 클래스의 다른 (아마도 동일한) 모델이 존재하는 경우, 모델 클래스는 시간 워프에 불변(invariant)이라고 말합니다.

이것은 잠재적으로 매우 유용한 속성입니다. 단기 종속성을 잘 모델링할 수 있는 RNN 클래스가 있고, 이 클래스가 시간 왜곡에 불변(invariant)임을 보여줄 수 있다면, 장기 종속성(단기 종속성을 가진 신호의 시간 팽창 왜곡(time dilation warping)에 해당)도 유용하게 포착할 수 있는 방식으로 이러한 모델을 훈련할 수 있다는 것을 알 수 있습니다. 곧 살펴보겠지만, 장거리 종속성을 모델링하기 위해 LSTM과 같은 게이트(gated) RNN 모델이 제안된 것은 우연이 아닙니다. 시간 왜곡 불변성(time warping invariance)을 달성하는 것은 LSTM의 입력/망각/출력(input/forget/output) 게이트와 같은 게이팅 메커니즘의 존재와 밀접하게 연관되어 있습니다.

시간이 $τ$만큼 뒤틀릴 때, $t$ 시점에 RNN이 관측한 신호는 $z(τ (t))$이며, 이러한 뒤틀림에 대해 불변성(invariant)을 유지하려면 동일하게 뒤틀린 요약 함수(equivalently-warped summary function) $h(τ (t))$를 예측해야 합니다. 테일러 확장 인수(Taylor expansion arguments)를 다시 한 번 사용하여 워프된 시간에 대한 방정식 50의 형태를 도출하고, RNN 업데이트 $\mathcal{R}$이 다음을 만족해야 합니다:

$$
    \frac{dh(τ (t))}{dτ (t)} = \mathcal{R} (z(τ (t + 1)), h(τ (t))) − h(τ (t))
\tag{51}
$$

그러나 위의 미분은 왜곡된 시간 $τ(t)$에 대해 계산되므로, 원래 신호는 고려하지 않습니다. 모델에 워핑 변환을 명시적으로 고려하려면, 워핑 요약 함수를 t에 대해 미분해야 합니다. 연쇄 규칙을 적용하면 다음과 같은 미분 방정식을 구할 수 있습니다: 

$$
    \frac{dh(τ (t))}{dt} = \frac{dh(τ (t))}{dτ (t)} \frac{dτ (t)}{dt} = \frac{dτ (t)}{dt} \mathcal{R} (z(τ (t + 1)), h(τ (t))) - \frac{dτ(t)}{dt} h(τ (t))
\tag{52}
$$

그리고 (연속 시간) RNN이 어떤 시간 왜곡 $τ(t)$에 대해서도 불변성(invariant)을 유지하려면, 미리 알고 있다고 가정할 수 없는 미분 $\frac{dτ(t)}{dt}$를 명시적으로 표현할 수 있어야 합니다! 이 미분을 근사화하는 학습 가능한(learnable) 함수 $Γ$를 도입해야 합니다. 예를 들어, $Γ$는 $z(t + 1)$과 $h(t)$를 고려하고 스칼라 출력을 예측하는 신경망일 수 있습니다.

이제 시간 왜곡을 받는 이산(discrete) RNN 모델의 관점에서 볼 때, 입력 $\mathbf{z}^{(t)}$는 $z(τ(t))$에 해당하고, 요약 $\mathbf{h}^{(t)}$는 $h(τ(t))$에 해당한다는 점에 주목합니다. 시간 왜곡에 대해 불변성(invariant)을 유지하기 위해 $\mathbf{h}^{(t)}$와 $\mathbf{h}^{(t+1)}$의 필수 관계(required relationship)를 구하기 위해, $h(τ(t))$의 1단계 테일러 확장(one-step Taylor
expansion )을 사용합니다:

$$
    h(τ (t + δ)) ≈ h(τ (t)) + δ \frac{dh(τ (t))}{dt}
$$

그리고 다시 한 번 $δ = 1$로 설정하고 방정식 52를 대입한 다음 이산합니다:

$$
\begin{align}
    \mathbf{h}^{(t+1)} & = \mathbf{h}^{(t)} + \frac{dτ(t)}{dt} \mathcal{R} (\mathbf{z}^{(t+1)}, \mathbf{h}^{(t)}) - \frac{dτ(t)}{dt} \mathbf{h}^{(t)}
\\
    & = \frac{dτ(t)}{dt} \mathcal{R} (\mathbf{z}^{(t+1)}, \mathbf{h}^{(t)}) + \left(1- \frac{dτ(t)}{dt} \right) \mathbf{h}^{(t)}
\end{align}
$$

마지막으로, 앞서 언급한 학습 가능한 함수인 $Γ$로 $\frac{dτ(t)}{dt}$를 바꿉니다. 이렇게 하면 시간 왜곡 불변(time warping-invariant) RNN에 필요한 형태가 완성됩니다: 

$$
    \mathbf{h}^{(t+1)} = Γ(\mathbf{z}^{(t+1)}, \mathbf{h}^{(t)})\mathcal{R} (\mathbf{z}^{(t+1)}, \mathbf{h}^{(t)}) + (1-Γ(\mathbf{z}^{(t+1)}, \mathbf{h}^{(t)}))\mathbf{h}^{(t)} \tag{53}
$$

방정식 53의 두 번째 항이 없다는 점을 감안할 때, SimpleRNN(방정식 39)은 시간 왜곡 불변이 아니라는 것(not time warping invariant)을 쉽게 추론할 수 있습니다. 대신, 시간 왜곡이 전혀 없다고 가정하는 경우, 즉 $\frac{dτ(t)}{dt} = 1$, 즉 $τ(t) = t$, 라고 가정하는 경우에 해당하는 $\mathcal{R} (\mathbf{z}^{(t+1)}, \mathbf{h}^{(t)})$로 $\mathbf{h}^{(t)}$를 완전히 덮어씁니다.

또한, 연속 시간(continuous-time) RNN과 R에 기반한 이산(discrete) RNN 간의 연결은 테일러 근사치의 정확도에 달려 있는데, 이는 시간 왜곡 도함수(time-warping derivative)가 너무 크지 않은 경우에만 유지됩니다(즉, $\frac{dτ(t)}{dt}  \lesssim 1$. 이에 대한 직관적인 설명은: 타임 워핑 작업이 중간 데이터 변경이 샘플링되지 않을 정도로 시간 증분(increments)$(t → t + 1)$을 크게 만드는 방식으로 시간을 수축(contracts time)시키는 경우, 모델은 타임 워핑된 입력을 원래 입력과 동일한 방식으로 처리할 수 없으며, 동일한 정보에 액세스할 수 없게 된다는 것입니다. 반대로, 모든 형태의 시간 팽창(dilations)(불연속적인 용어로, 입력 시계열에 0이 산재하는 것에 해당)은 프레임워크 내에서 완벽하게 허용됩니다.

단조롭게 증가하는 $τ(\frac{dτ(t)}{dt} > 0)$라는 요구 사항과 결합하면, $Γ$의 출력 공간을 $0 < Γ(\mathbf{z}^{(t+1)}, \mathbf{h}^{(t)}) < 1$로 제한할 수 있으며, 이는 $Γ$에 로지스틱 시그모이드(logistic sigmoid) 활성화를 사용하도록 동기를 부여합니다:

$$
    Γ(\mathbf{z}^{(t+1)}, \mathbf{h}^{(t)}) = \text{logistic} (\mathbf{W}_Γ\mathbf{z}^{(t+1)} + \mathbf{U}_Γ \mathbf{h}^{(t)} + \mathbf{b}_Γ)
$$

즉 위 식은 LSTM 게이팅 방정식(예: 방정식 44)과 정확히 일치합니다. 가장 큰 차이점은 LSTM은 게이팅 *벡터*를 계산하는 반면, 방정식 53은 $Γ$가 스칼라를 출력해야 한다는 것을 암시한다는 것입니다. 벡터화된 게이트(Vectorised gates)(Hochreiter, 1991)는 $\mathbf{h}^{(t)}$의 모든 차원에 다른 뒤틀림 미분을 맞출 수 있으므로, 여러 시간대에 걸쳐 동시에 추론할 수 있습니다. 

여기서 잠시 멈춰서 우리가 한 일을 요약할 필요가 있습니다. RNN 클래스가 (비파괴적) 시간 왜곡에 대해 불변(invariant)임을 요구함으로써, 우리는 그것이 가져야 하는 필수 형태를 도출했고(방정식 53), 이것이 게이트(gated) RNN의 클래스와 정확히 일치한다는 것을 보여주었습니다. 이 관점에서 게이트의 주요 역할은 워핑 변환의 미분 $\frac{dτ(t)}{dt}$를 정확하게 맞추는 것입니다.

클래스 불변성(class invariance)의 개념은 앞서 공부한 불변성과는 다소 다릅니다. 즉, $τ_1(t)$로 타임워프된 입력에 대해 게이트 RNN을 훈련한 후에는, 일반적으로 다른 $τ_2(t)$로 워프된 신호로 제로샷 전송할 수 없습니다. 오히려 클래스 불변성은 게이트 RNN이 동일한 방식으로 두 신호를 모두 맞출(fit) 수 있을 만큼 강력하지만, 모델 파라미터가 크게 다를 수 있다는 것을 보장할 뿐입니다. 즉, 효과적인 게이팅 메커니즘이 뒤틀림 미분(warping derivative)을 맞추는 것과 밀접한 관련이 있다는 사실을 깨닫게 되면, 이제 간략하게 설명할 것처럼 게이트 RNN 최적화를 위한 유용한 처방을 얻을 수 있습니다. 

>제로 샷(zero-shot) 전송이 불가능한 한 가지 경우는, 두 번째 시간 워프가 첫 번째 시간 워프의 시간 재조정(time rescaling)$(τ_2(t) = ατ_1(t))$이라고 가정할 때입니다.

>$τ_1$에서 미리 훈련된 게이트 RNN을 $τ_2$로 워프된 신호로 전송하려면, 게이트의 스케일을 다시 조정(rescaling the gates)하기만 하면 됩니다: $Γ_2(\mathbf{z}^{(t+1)} , \mathbf{h}^{(t)}) = αΓ_1(\mathbf{z}^{(t+1)} , \mathbf{h}^{(t)})$. $R$은 파라미터를 유지할 수 있습니다$(R_1 = R_2)$.

예를 들어, 신호 내에서 추적하고자 하는 종속성의 범위가 $[T_l , T_h]$ 타임스텝 범위라고 가정할 수 있습니다.

해석적 해(analytic solutions)를 방정식 52로 분석하면, 게이팅된 RNN의 특징적인 망각 시간(forgetting time) $\mathbf{h}^{(t)}$는 $\frac{1}{Γ(\mathbf{z}^{(t+1)}, \mathbf{h}^{(t)})}$에 비례한다는 것을 알 수 있습니다. 따라서 가정한 범위 내에서 정보를 효과적으로 기억하려면 게이팅 값이 $\left[ \frac{1}{T_h} \frac{1}{T_m}  \right]$사이에 있기를 원합니다.

또한 레이어 정규화와 같은 변환을 적용할 때 흔히 발생하는 부산물인 $\mathbf{z}^{(t)}$와 $\mathbf{h}^{(t)}$가 대략 영 중심(zero-centered)이라고 가정하면(Ba et al., 2016), $\mathbb{E}[Γ(\mathbf{z}^{(t+1)}, \mathbf{h}^{(t)})] ≈ \text{logistic}(\mathbf{b}_Γ)$으로 가정할 수 있습니다. 따라서 게이팅 메커니즘의 바이어스(bias) 벡터를 제어하는 것은 유효 게이트 값(effective gate value)을 제어하는 매우 강력한 방법입니다.

>이러한 인사이트는 이미 Gers와 Schmidhuber(2000)에 의해 발견되었으며, Jozefowicz 등(2015)은 LSTM의 망각 게이트 바이어스(forget-gate bias)를 1과 같은 일정한 양의 벡터(constant positive vector)로 초기화할 것을 경험적으로 권장했습니다. 

이 두 가지 관찰 결과를 결합하면, $\mathbf{b}_Γ∼ - \log(\mathcal{U}(T_l , T_h)-1)$로 초기화하면 적절한 범위의 게이팅 값을 얻을 수 있다는 결론을 내릴 수 있으며, 여기서 $\mathcal{U}$는 균일한 실수 분포입니다. 이러한 권장 사항은 Tallec과 올리비에(Tallec and Ollivier)(2018)에 의해 크로노 초기화(chrono initialisation)라고 불렸으며, 경험적으로 게이팅된 RNN의 장거리 종속성 모델링을 개선하는 것으로 입증되었습니다. 

### RNN을 사용한 시퀀스 간 학습
Sequence-to-sequence learning with RNNs

RNN 기반 계산을 사용한 대표적인 역사적 사례는 자연어 기계 번역과 같은 시퀀스 간 (sequence-to-sequence) 번역 작업입니다. Sutskever 등(2014)의 선구적인 seq2seq 작업은 요약 벡터인 $\mathbf{h}^{(T)}$를 디코더 RNN의 초기 입력으로 전달하고, 다음 단계의 입력으로 RNN 블록의 출력을 제공함으로써 이를 달성했습니다.

이로 인해 요약 벡터인 $\mathbf{h}^{(T)}$에 상당한 표현 압력(representational pressure)이 가해졌습니다. 딥러닝의 맥락에서, $\mathbf{h}^{(T)}$는 병목 현상(bottleneck)이라고도 합니다. 고정된 용량은 해당 시퀀스를 생성하는 데 도움이 되는 방식으로 전체 입력 시퀀스의 내용을 표현하기에 충분해야 하며, 길이가 상당히 다른 입력 시퀀스도 지원해야 합니다(그림 21).

>병목 효과는 최근 그래프 표현 학습 커뮤니티(Alon and Yahav, 2020)와 신경 알고리즘 추론(Cappart et al., 2021)에서 상당한 주목을 받고 있습니다. 

실제로는, 출력의 다른(different) 단계가 입력의 다른 부분에 집중(주의(attend))하기를 원할 수 있으며, 이러한 모든 선택은 병목 벡터를 통해 표현하기 어렵습니다. 이러한 관찰에 따라, 널리 사용되는 반복적 주의(recurrent attention) 모델은 Bahdanau 등(2014)에 의해 제안되었습니다. 처리의 모든 단계에서, 쿼리 벡터(query vector)가 RNN에 의해 생성되고, 이 쿼리 벡터는 주로 가중 합계를 계산하여 모든 시간 단계 $\mathbf{h}^{(t)}$의 표현과 상호 작용합니다. 이 모델은 신경 콘텐츠 기반 주의력(neural content-based attention)을 개척했으며 트랜스포머 모델의 성공보다 앞서 있습니다.

마지막으로, 어텐딩은 입력 콘텐츠의 일부에 동적으로 집중할 수 있는 부드러운(soft) 방법을 제공하지만, 상당한 연구를 통해 입력에 주의를 집중하는 보다 명시적인(explicit) 방법도 학습했습니다. 알고리즘에 기반한 강력한 방법으로는 Vinyals 등(2015)의 포인터 네트워크(pointer network)가 있는데, 이 네트워크는 가변 크기(variable-sized)의 입력 요소를 가리킬 수 있도록 반복적 주의를 간단히 수정할 것을 제안합니다. 이러한 연구 결과는, 포인터 네트워크 지원 LSTM이 지원하는 정렬되지 않은 집합에 대해 seq2seq 모델을 일반화하는 set2set 아키텍처(Vinyals et al., 2016)로 일반화되었습니다.